In [2]:
# from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2 ##STrAdaBoost.R2
# from TwoStageTrAdaBoostR2 import TwoStageTrAdaBoostR2 ##two-stage TrAdaBoost.R2

import pandas as pd
import sys
import numpy as np
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dense, Activation, Conv2D, Dropout, Flatten
from keras import optimizers, utils, initializers, regularizers
import keras.backend as K

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler #Importing the StandardScaler

from itertools import combinations

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats.stats import pearsonr

from math import sqrt

#Geo plotting libraries
import geopandas as gdp
from matplotlib.colors import ListedColormap
# import geoplot as glpt

import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import KFold
import matplotlib.lines as mlines

######### Instance Transfer repositories ####################
from adapt.instance_based import TwoStageTrAdaBoostR2

print("Repositories uploaded!!")

Repositories uploaded!!


In [3]:
from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2
from sklearn.model_selection import GridSearchCV
from adapt.instance_based import KMM

print("Second Upload Completed!!")

Second Upload Completed!!


In [87]:
################################### Concrete ###########################################################################################################
ConcreteData_df = pd.read_excel('UCI_regression/Concrete/Concrete_Data.xls') ## 'Cement' found to be correlated at 0.4 :: 100
print("Concrete Data")
print("-------------------------------------------")
print(ConcreteData_df.shape)

drop_col_concrete = ['Cement']

concrete_tgt_df = ConcreteData_df.loc[(ConcreteData_df['Cement'] <= 225)]
concrete_tgt_df = concrete_tgt_df.drop(drop_col_concrete, axis = 1)
concrete_tgt_df = concrete_tgt_df.reset_index(drop=True)
print("Target Set: ",concrete_tgt_df.shape)

concrete_source_df = ConcreteData_df.loc[(ConcreteData_df['Cement'] > 225) & (ConcreteData_df['Cement'] <= 350)]
concrete_source_df = concrete_source_df.drop(drop_col_concrete, axis = 1)
concrete_source_df = concrete_source_df.reset_index(drop=True)
print("Source Set: ",concrete_source_df.shape)

concrete_test_df = ConcreteData_df.loc[(ConcreteData_df['Cement'] > 350)]
concrete_test_df = concrete_test_df.drop(drop_col_concrete, axis = 1)
concrete_test_df = concrete_test_df.reset_index(drop=True)
print("Test Set: ",concrete_test_df.shape)

#################### Splitting into features and target ####################
target_column_concrete = ['ConcreteCompressiveStrength']

concrete_tgt_df_y = concrete_tgt_df[target_column_concrete]
concrete_tgt_df_X = concrete_tgt_df.drop(target_column_concrete, axis = 1)

concrete_source_df_y = concrete_source_df[target_column_concrete]
concrete_source_df_X = concrete_source_df.drop(target_column_concrete, axis = 1)

concrete_test_df_y = concrete_test_df[target_column_concrete]
concrete_test_df_X = concrete_test_df.drop(target_column_concrete, axis = 1)

############## Merging the datasets ##########################################
concrete_X_df = pd.concat([concrete_tgt_df_X, concrete_source_df_X], ignore_index=True)
concrete_y_df = pd.concat([concrete_tgt_df_y, concrete_source_df_y], ignore_index=True)

concrete_np_train_X = concrete_X_df.to_numpy()
concrete_np_train_y = concrete_y_df.to_numpy()

concrete_np_test_X = concrete_test_df_X.to_numpy()
concrete_np_test_y = concrete_test_df_y.to_numpy()

concrete_np_train_y_list = concrete_np_train_y.ravel()
concrete_np_test_y_list = concrete_np_test_y.ravel()

src_size_concrete = len(concrete_source_df_y)
tgt_size_concrete = len(concrete_tgt_df_y)

print("-------------------------------------------")

################################################################################################################################################################

Concrete Data
-------------------------------------------
(1030, 9)
Target Set:  (368, 8)
Source Set:  (406, 8)
Test Set:  (256, 8)
-------------------------------------------


In [5]:
############# Transfer Learning specifications #########################
src_idx = np.arange(start=0, stop=(src_size_concrete - 1), step=1)
tgt_idx = np.arange(start=src_size_concrete, stop=((src_size_concrete + tgt_size_concrete)-1), step=1)


def get_estimator(**kwargs):
    return DecisionTreeRegressor(max_depth = 6)

kwargs_TwoTrAda = {'steps': 30,
                    'fold': 10,
                  'learning_rate': 0.1}

In [6]:
########################### TwoStagetrAdaBoostR2 #######################################

model_TwoTrAda_concrete = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 1000, cv=10) #, kwargs_TwoTrAda)
model_TwoTrAda_concrete.fit(concrete_np_train_X, concrete_np_train_y_list, src_idx, tgt_idx)

print("Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

y_pred_TwoTrAda_concrete = model_TwoTrAda_concrete.predict(concrete_np_test_X)
mse_TwoTrAda_concrete = sqrt(mean_squared_error(concrete_np_test_y, y_pred_TwoTrAda_concrete))
print("RMSE of Two-Stage TrAdaboost.R2:", mse_TwoTrAda_concrete)

r2_score_TwoTrAda_concrete = pearsonr(concrete_np_test_y_list, y_pred_TwoTrAda_concrete)
r2_score_TwoTrAda_concrete = (r2_score_TwoTrAda_concrete[0])**2
print("R^2 of TrAdaboostR2:", r2_score_TwoTrAda_concrete)

print("-------------------------------------------")

Two-Stage TrAdaboost.R2
-------------------------------------------
RMSE of Two-Stage TrAdaboost.R2: 14.541708818834294
R^2 of TrAdaboostR2: 0.6380025091352669
-------------------------------------------


In [7]:
########################### AdaBoostR2 Transfer LearningL #####################################################
print("Adaboost.R2 Transfer Learning")
print("-------------------------------------------")


model_Ada_concrete = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 8), learning_rate=0.01, n_estimators=500) 
model_Ada_concrete.fit(concrete_np_train_X, concrete_np_train_y_list)

y_pred_ada_concrete = model_Ada_concrete.predict(concrete_np_test_X)
mse_Ada_concrete = sqrt(mean_squared_error(concrete_np_test_y, y_pred_ada_concrete))
print("RMSE of Adaboost.R2:", mse_Ada_concrete)

r2_score_Ada_concrete = pearsonr(concrete_np_test_y_list, y_pred_ada_concrete)
r2_score_Ada_concrete = (r2_score_Ada_concrete[0])**2
print("R^2 of TrAdaboostR2:", r2_score_Ada_concrete)

print("-------------------------------------------")

Adaboost.R2 Transfer Learning
-------------------------------------------
RMSE of Adaboost.R2: 14.14134604172718
R^2 of TrAdaboostR2: 0.6617768705437826
-------------------------------------------


In [8]:
########################### AdaBoostR2 #####################################################
print("Adaboost.R2")
print("-------------------------------------------")


model_Ada_concrete = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 8), learning_rate=0.01, n_estimators=500) 
model_Ada_concrete.fit(concrete_tgt_df_X, concrete_tgt_df_y)

y_pred_ada_concrete = model_Ada_concrete.predict(concrete_np_test_X)
mse_Ada_concrete = sqrt(mean_squared_error(concrete_np_test_y, y_pred_ada_concrete))
print("RMSE of Adaboost.R2:", mse_Ada_concrete)

r2_score_Ada_concrete = pearsonr(concrete_np_test_y_list, y_pred_ada_concrete)
r2_score_Ada_concrete = (r2_score_Ada_concrete[0])**2
print("R^2 of TrAdaboostR2:", r2_score_Ada_concrete)

print("-------------------------------------------")

Adaboost.R2
-------------------------------------------


/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


RMSE of Adaboost.R2: 24.871232748488882
R^2 of TrAdaboostR2: 0.4313630624407841
-------------------------------------------


In [9]:
######################### Kernel Mean Matching #######################################
print("Kernel Mean Matching")
print("-------------------------------------------")

model_KMM_concrete = KMM(get_estimator = get_estimator)
model_KMM_concrete.fit(concrete_np_train_X, concrete_np_train_y_list, src_idx, tgt_idx)

y_pred_KMM_concrete = model_KMM_concrete.predict(concrete_test_df_X) ##Using dataframe instead of the numpy matrix
mse_KMM_concrete = sqrt(mean_squared_error(concrete_np_test_y, y_pred_KMM_concrete))
print("RMSE of KMM:", mse_KMM_concrete)

r2_score_KMM_concrete = pearsonr(concrete_np_test_y_list, y_pred_KMM_concrete)
r2_score_KMM_concrete = (r2_score_KMM_concrete[0])**2
print("R^2 of TrAdaboostR2:", r2_score_KMM_concrete)

#model_KMM_concrete.weights_   #To get the weights of the algorithm

print("-------------------------------------------")

Kernel Mean Matching
-------------------------------------------
RMSE of KMM: 24.695170122194522
R^2 of TrAdaboostR2: 0.2077368001621871
-------------------------------------------


In [10]:
######################### Gradient Boosting Regression Transfer Learning #######################################

print("Gradient Boosting Regression Transfer Learning")
print("-------------------------------------------")


model_GBR_concrete = GradientBoostingRegressor(learning_rate=0.01, max_depth=4, n_estimators=1000, subsample=0.5)
model_GBR_concrete.fit(concrete_np_train_X, concrete_np_train_y_list)

y_pred_GBR_concrete = model_GBR_concrete.predict(concrete_test_df_X) ##Using dataframe instead of the numpy matrix
mse_GBR_concrete = sqrt(mean_squared_error(concrete_np_test_y, y_pred_GBR_concrete))
print("RMSE of KMM:", mse_GBR_concrete)

r2_score_GBR_concrete = pearsonr(concrete_np_test_y_list, y_pred_GBR_concrete)
r2_score_GBR_concrete = (r2_score_GBR_concrete[0])**2
print("R^2 of TrAdaboostR2:", r2_score_GBR_concrete)

print("-------------------------------------------")

Gradient Boosting Regression Transfer Learning
-------------------------------------------
RMSE of KMM: 11.258851937503154
R^2 of TrAdaboostR2: 0.6895184459321161
-------------------------------------------


In [11]:
######################### Gradient Boosting Regression #######################################

print("Gradient Boosting Regression")
print("-------------------------------------------")


model_GBR_concrete = GradientBoostingRegressor(learning_rate=0.01, max_depth=4, n_estimators=1000, subsample=0.5)
model_GBR_concrete.fit(concrete_tgt_df_X, concrete_tgt_df_y)

y_pred_GBR_concrete = model_GBR_concrete.predict(concrete_test_df_X) ##Using dataframe instead of the numpy matrix
mse_GBR_concrete = sqrt(mean_squared_error(concrete_np_test_y, y_pred_GBR_concrete))
print("RMSE of KMM:", mse_GBR_concrete)

r2_score_GBR_concrete = pearsonr(concrete_np_test_y_list, y_pred_GBR_concrete)
r2_score_GBR_concrete = (r2_score_GBR_concrete[0])**2
print("R^2 of TrAdaboostR2:", r2_score_GBR_concrete)

print("-------------------------------------------")

Gradient Boosting Regression
-------------------------------------------


/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


RMSE of KMM: 23.41655933034992
R^2 of TrAdaboostR2: 0.5650871647106431
-------------------------------------------


In [13]:
################ Transformed TrAdaBoost.R2 #############################################

weights_KMM = model_KMM_concrete.weights_

val_mean = np.mean(weights_KMM)
weights_KMM = np.append(weights_KMM, val_mean)

concrete_source_df_X_trans = concrete_source_df_X.apply(lambda concrete_source_df_X: concrete_source_df_X * weights_KMM)
#concrete_source_df_X.multiply(weights_KMM, axis=0)

# concrete_np_train_X_trans = concrete_source_df_X_trans.to_numpy()

concrete_X_df = pd.concat([concrete_tgt_df_X, concrete_source_df_X_trans], ignore_index=True)
concrete_y_df = pd.concat([concrete_tgt_df_y, concrete_source_df_y], ignore_index=True)

concrete_np_train_X = concrete_X_df.to_numpy()
concrete_np_train_y = concrete_y_df.to_numpy()

concrete_np_test_X = concrete_test_df_X.to_numpy()
concrete_np_test_y = concrete_test_df_y.to_numpy()

concrete_np_train_y_list = concrete_np_train_y.ravel()
concrete_np_test_y_list = concrete_np_test_y.ravel()


# print(concrete_np_train_y_list.shape, concrete_np_train_X_trans.shape)

##################################################################
model_TwoTrAda_concrete = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 100, cv=10) #, kwargs_TwoTrAda)
model_TwoTrAda_concrete.fit(concrete_np_train_X, concrete_np_train_y_list, src_idx, tgt_idx)

print("Transformed Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

y_pred_TwoTrAda_concrete = model_TwoTrAda_concrete.predict(concrete_np_test_X)

mse_TwoTrAda_concrete = sqrt(mean_squared_error(concrete_np_test_y, y_pred_TwoTrAda_concrete))
print("RMSE of Two-Stage TrAdaboost.R2:", mse_TwoTrAda_concrete)

r2_score_TwoTrAda_concrete = pearsonr(concrete_np_test_y_list, y_pred_TwoTrAda_concrete)
r2_score_TwoTrAda_concrete = (r2_score_TwoTrAda_concrete[0])**2
print("R^2 of TrAdaboostR2:", r2_score_TwoTrAda_concrete)

print("-------------------------------------------")

Transformed Two-Stage TrAdaboost.R2
-------------------------------------------
RMSE of Two-Stage TrAdaboost.R2: 20.34869140770156
R^2 of TrAdaboostR2: 0.3977255550270938
-------------------------------------------


In [89]:
#################################### STrAdaBoost.R2 ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

sample_size = [len(concrete_tgt_df_X), len(concrete_source_df_X)]
n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

model_stradaboost_concrete = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6),
                    n_estimators = n_estimators, sample_size = sample_size,
                    steps = steps, fold = fold, random_state = random_state)


model_stradaboost_concrete.fit(concrete_np_train_X, concrete_np_train_y_list)
y_pred_stradaboost_concrete = model_stradaboost_concrete.predict(concrete_np_test_X)

mse_stradaboost_concrete = sqrt(mean_squared_error(concrete_np_test_y, y_pred_stradaboost_concrete))
print("RMSE of STrAdaboostR2:", mse_stradaboost_concrete)

r2_score_stradaboost_concrete = pearsonr(concrete_np_test_y_list, y_pred_stradaboost_concrete)
r2_score_stradaboost_concrete = (r2_score_stradaboost_concrete[0])**2
print("R^2 of STrAdaboostR2:", r2_score_stradaboost_concrete)

Inside STrAdaBoost.R2
RMSE of STrAdaboostR2: 12.86329037846698
R^2 of STrAdaboostR2: 0.7153926855970645


In [ ]:
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2
# from newtwoStage_TrAdaBoostR2 import TradaboostRegressor

import pandas as pd
import sys
import numpy as np
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dense, Activation, Conv2D, Dropout, Flatten
from keras import optimizers, utils, initializers, regularizers
import keras.backend as K

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler #Importing the StandardScaler

from itertools import combinations

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats.stats import pearsonr
from math import sqrt

################# Geo plotting libraries
import geopandas as gdp
from matplotlib.colors import ListedColormap
# import geoplot as glpt

import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from sklearn.ensemble import GradientBoostingRegressor

from statistics import mean
from sklearn.cluster import KMeans
from scipy.spatial import distance
import statistics

from matplotlib.transforms import Affine2D
import folium
import matplotlib.lines as mlines

###################################################################################################################################################################
STrAdaBoost_RMSE_NOx = [84.49069754636324, 83.19323870128466, 81.27928359120179, 82.18747206693598, 82.60804181502738, 82.65985343387109,
81.93233750015905, 83.74998065696522, 82.92239432705983, 84.71932508116717]
TrAdaBoost_RMSE_NOx = [76.29870110031176, 90.49733765158976, 88.66740065772586, 76.61069182772832, 89.19167720613721, 76.63572940793705,
75.91621027222305, 88.91209884411252, 75.66965212493261,76.56390041476169]
AdaBoost_RMSE_NOx = [90.63932170294021, 87.15426790901572, 88.34473432945258, 86.93531815065907, 86.09325821106664, 90.33874900883123,
89.75706293974721, 89.18255026677808, 86.79257145189278,88.15379488581348]

STrAdaBoost_RMSE_NO2 = [24.283690808360245, 24.127661779305818, 23.795924043958564, 23.86071279600051, 23.842607206027456, 23.678043189310404,
24.832202009871946, 24.72540998123888, 23.952057769016328, 24.499915240375014]
TrAdaBoost_RMSE_NO2 = [29.967565846373816, 29.918410405444156, 30.07324513362625, 30.057292629368256, 30.15716067372586, 30.099997992631963,
30.169099696332275, 30.116280516373376, 29.989850789314207, 30.29776937245375]
AdaBoost_RMSE_NO2 = [27.587178104589203, 27.64482357816312, 26.6654992063022, 26.949370826873007, 27.357169025084616, 27.438634766666453,
28.137109920599684, 27.36063648100872, 27.929103697001345, 28.52297816586722]

STrAdaBoost_RMSE_O3 = [173.64151805705194, 173.6334619111963, 173.65662665726717, 174.1308577850406, 174.0310122302578, 174.07118389774956, 173.3810950553901,
173.7619765646141, 173.89517540044486, 173.71213163191527]
TrAdaBoost_RMSE_O3 = [171.6515413016132, 170.14551538005634, 170.69377563176536, 170.23152266767838, 172.05586024289434, 169.06705817239322, 173.20665423910825,
172.54867998954228, 170.27823788650394, 170.2347615205619]
AdaBoost_RMSE_O3 = [180.68119379123922, 177.1889644285649, 178.03999086375444, 177.73683263644602, 178.61357778268032, 178.53935080389707, 180.62098338916542,
180.12605079594596, 179.4711714920254, 182.32380075254858]

# STrAdaBoost_RMSE_NOx_Norm = [116.56207487781757, 117.12744898440695, 122.21001928176337, 116.02213691302707, 115.01356572936842, 117.82206454755801, 116.33390399761493,
# 117.29859060209448, 117.15492417635203, 118.52620461003417]
# TrAdaBoost_RMSE_NOx = [135.32110275619965, 134.39566747250927, 134.72552610715496, 133.4738476573543, 136.75743147426928, 136.6462951739915, 134.9491286623907,
# 134.6030488331259, 134.27917844884996,135.24583900366477]
# AdaBoost_RMSE_NOx = [87.51229327934197, 88.32432900140913, 88.08834819883502, 87.52912955139539, 88.6824868839292, 88.11432378088877, 86.51542456695174,
# 88.2780125086002, 88.42890140495514, 85.80770659201272]
#
# STrAdaBoost_RMSE_NO2_Norm = [45.623681500346684, 45.627289088592484, 45.6410716215836, 45.950142322965014, 45.746781764008034, 45.51898933766781, 46.416842540407266,
# 45.61379839353198, 45.46275490362647, 45.48011193000623]
# TrAdaBoost_RMSE_NO2 = [44.74193223206197, 44.65640007857625, 44.7477254703826, 44.74501305364314, 44.80431951909328, 44.74216239294229, 44.65573169849547,
# 44.81223332738691, 44.59243606389392, 44.66763885614316]
# AdaBoost_RMSE_NO2 = [27.386627364748396, 27.91727196005653, 27.064526060877395, 26.934118784173428, 28.273217386873775, 27.57585539705687, 27.490141344356058,
# 28.16892932308915, 27.31365610561806, 27.90207530790275]

############################################################################################################################################################

methods = ['S-TrAdaBoost.R2', 'two-stage TrAdaBoost.R2', 'AdaBoost.R2']

# STrAdaBoost_R2_val = (max(STrAdaBoost_R2) - min(STrAdaBoost_R2))/2
# AdaBoost_R2_val = (max(AdaBoost_R2) - min(AdaBoost_R2))/2
# TrAdaBoost_R2_val = (max(TrAdaBoost_R2) - min(TrAdaBoost_R2))/2

STrAdaBoost_std1 = statistics.pstdev(STrAdaBoost_RMSE_NOx)
TrAdaBoost_std1 = statistics.pstdev(TrAdaBoost_RMSE_NOx)
AdaBoost_std1 = statistics.pstdev(AdaBoost_RMSE_NOx)

STrAdaBoost_std2 = statistics.pstdev(STrAdaBoost_RMSE_NO2)
TrAdaBoost_std2 = statistics.pstdev(TrAdaBoost_RMSE_NO2)
AdaBoost_std2 = statistics.pstdev(AdaBoost_RMSE_NO2)

STrAdaBoost_std3 = statistics.pstdev(STrAdaBoost_RMSE_O3)
TrAdaBoost_std3 = statistics.pstdev(TrAdaBoost_RMSE_O3)
AdaBoost_std3 = statistics.pstdev(AdaBoost_RMSE_O3)

# STrAdaBoost_std3 = statistics.pstdev(STrAdaBoost_R2_NOx_Norm)
# TrAdaBoost_std3 = statistics.pstdev(TrAdaBoost_R2_NOx_Norm)
# AdaBoost_std3 = statistics.pstdev(AdaBoost_R2_NOx_Norm)
#
# STrAdaBoost_std4 = statistics.pstdev(STrAdaBoost_R2_NOx_Norm)
# TrAdaBoost_std4 = statistics.pstdev(TrAdaBoost_R2_NOx_Norm)
# AdaBoost_std4 = statistics.pstdev(AdaBoost_R2_NOx_Norm)


y1 = [mean(STrAdaBoost_RMSE_NOx), mean(TrAdaBoost_RMSE_NOx), mean(AdaBoost_RMSE_NOx)]
dy1 = [STrAdaBoost_std1, TrAdaBoost_std1, AdaBoost_std1]


y2 = [mean(STrAdaBoost_RMSE_NO2), mean(TrAdaBoost_RMSE_NO2), mean(AdaBoost_RMSE_NO2)]
dy2 = [STrAdaBoost_std2, TrAdaBoost_std2, AdaBoost_std2]

y3 = [mean(STrAdaBoost_RMSE_O3), mean(TrAdaBoost_RMSE_O3), mean(AdaBoost_RMSE_O3)]
dy3 = [STrAdaBoost_std3, TrAdaBoost_std3, AdaBoost_std3]

# y3 = [mean(STrAdaBoost_R2_NOx_Norm), mean(TrAdaBoost_R2_NOx_Norm), mean(AdaBoost_R2_NOx_Norm)]
# dy3 = [STrAdaBoost_std3, TrAdaBoost_std3, AdaBoost_std3]
#
# y4 = [mean(STrAdaBoost_R2_NO2_Norm), mean(TrAdaBoost_R2_NO2_Norm), mean(AdaBoost_R2_NO2_Norm)]
# dy4 = [STrAdaBoost_std4, TrAdaBoost_std4, AdaBoost_std4]

plus = mlines.Line2D([], [], marker='P', color='maroon', linestyle='None',
                          markersize = 20, label='NO2')
circle = mlines.Line2D([], [], marker='o', color='black', linestyle='None',
                          markersize = 20, label='NOx')
triangle = mlines.Line2D([], [], marker='v', color='blue', linestyle='None',
                          markersize = 20, label='O3')


x = np.arange(len(methods))

print(dy1)
print(dy2)
print(dy3)

fig, ax = plt.subplots()

# trans1 = Affine2D().translate(-0.025, 0.0) + ax.transData
# trans2 = Affine2D().translate(+0.025, 0.0) + ax.transData
# trans3 = Affine2D().translate(+0.05, 0.0) + ax.transData
# trans4 = Affine2D().translate(-1.5, 0.0) + ax.transData

# er1 = ax.errorbar(x, y1, yerr = dy1, marker = "o", linestyle = "None", transform = trans1, elinewidth = 10, markersize = 20)
# er2 = ax.errorbar(x, y2, yerr = dy2, marker = "s", linestyle = "None", transform = trans2, elinewidth = 6, markersize = 10)
# er3 = ax.errorbar(x, y3, yerr = dy3, marker = "o", linestyle = "None", transform = trans3)
# er4 = ax.errorbar(x, y4, yerr = dy4, marker = "o", linestyle = ":", transform = trans4)


plt.errorbar(x, y1, yerr = dy1, fmt='o', color='black', ecolor='black', elinewidth = 10, markersize = 20) #capsize=0)
plt.errorbar(x, y2, yerr = dy2, fmt='P', color='maroon', ecolor='maroon', elinewidth = 10, markersize = 20)
plt.errorbar(x, y3, yerr = dy3, fmt='v', color='blue', ecolor='blue', elinewidth = 10, markersize = 20) #capsize=0)
# plt.xticks(x)
plt.xticks(np.arange(3), ('S-TrAdaBoost.R2', 'two-stage TrAdaBoost.R2', 'AdaBoost.R2'), fontsize = 32)
plt.yticks(fontsize = 30)
plt.ylabel("RMS Error", fontsize = 36)
# plt.title('Mean RMSE for methodologies with Standard deviation error bars', fontsize=12)
plt.legend(handles= [plus, circle, triangle], fontsize = 20, loc= 'center left')

plt.show()

# fig, ax = plt.subplots()
# ax.bar(x, y, yerr=dy, align='center', alpha=0.5, ecolor='black', capsize=10)
# ax.yticks(np.arange(0.25, 0.40, 0.02))
# ax.set_ylabel('Coefficient of Thermal Expansion ($\degree C^{-1}$)')
# ax.set_xticks(x)
# ax.set_xticklabels(methods)
# ax.set_title('Coefficent of Thermal Expansion (CTE) of Three Metals')
# ax.yaxis.grid(True)
# plt.tight_layout()
# plt.show()

In [97]:
######################################################## Housing ################################################################
## 'nox' found to be correlated at 0.4 :: [0.385 - 0.871] :: 50
#################################################################################################################################
HousingData_df = pd.read_csv('UCI_regression/BostonHousing/BostonHousing.csv') 
print("Housing Data")
print(HousingData_df.shape)

drop_col_housing = ['nox']
housing_tgt_df = HousingData_df.loc[(HousingData_df['nox'] > 0.475) & (HousingData_df['nox'] <= 0.600)]
housing_tgt_df = housing_tgt_df.drop(drop_col_housing, axis = 1)
housing_tgt_df = housing_tgt_df.reset_index(drop=True)
print("Target Set: ",housing_tgt_df.shape)


housing_source_df = HousingData_df.loc[(HousingData_df['nox'] <= 0.475)]
housing_source_df = housing_source_df.drop(drop_col_housing, axis = 1)
housing_source_df = housing_source_df.reset_index(drop=True)
print("Source Set: ",housing_source_df.shape)


housing_test_df = HousingData_df.loc[(HousingData_df['nox'] > 0.600)]
housing_test_df = housing_test_df.drop(drop_col_housing, axis = 1)
housing_test_df = housing_test_df.reset_index(drop=True)
print("Test Set: ",housing_test_df.shape)

#################### Splitting into features and target ####################
target_column_housing = ['medv']

housing_tgt_df_y = housing_tgt_df[target_column_housing]
housing_tgt_df_X = housing_tgt_df.drop(target_column_housing, axis = 1)

housing_source_df_y = housing_source_df[target_column_housing]
housing_source_df_X = housing_source_df.drop(target_column_housing, axis = 1)

housing_test_df_y = housing_test_df[target_column_housing]
housing_test_df_X = housing_test_df.drop(target_column_housing, axis = 1)

############## Merging the datasets ##########################################
housing_X_df = pd.concat([housing_tgt_df_X, housing_source_df_X], ignore_index=True)
housing_y_df = pd.concat([housing_tgt_df_y, housing_source_df_y], ignore_index=True)

housing_np_train_X = housing_X_df.to_numpy()
housing_np_train_y = housing_y_df.to_numpy()

housing_np_test_X = housing_test_df_X.to_numpy()
housing_np_test_y = housing_test_df_y.to_numpy()

housing_np_train_y_list = housing_np_train_y.ravel()
housing_np_test_y_list = housing_np_test_y.ravel()

src_size_housing = len(housing_source_df_y)
tgt_size_housing = len(housing_tgt_df_y)

print("---------------------------")

Housing Data
(506, 14)
Target Set:  (186, 13)
Source Set:  (155, 13)
Test Set:  (165, 13)
---------------------------


In [98]:
######################## Transfer Learning specifications for Housing #########################################
src_idx_housing = np.arange(start=0, stop=(src_size_housing - 1), step=1)
tgt_idx_housing = np.arange(start=src_size_housing, stop=((src_size_housing + tgt_size_housing)-1), step=1)


def get_estimator(**kwargs):
    return DecisionTreeRegressor(max_depth = 6)

kwargs_TwoTrAda = {'steps': 30,
                    'fold': 10,
                  'learning_rate': 0.1}

In [95]:
########################### TwoStagetrAdaBoostR2 Housing #######################################

model_TwoTrAda_housing = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 1000, cv=10) #, kwargs_TwoTrAda)
model_TwoTrAda_housing.fit(housing_np_train_X, housing_np_train_y_list, src_idx_housing, tgt_idx_housing)

print("Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

y_pred_TwoTrAda_housing = model_TwoTrAda_housing.predict(housing_np_test_X)
mse_TwoTrAda_housing = sqrt(mean_squared_error(housing_np_test_y, y_pred_TwoTrAda_housing))
print("RMSE of Two-Stage TrAdaboost.R2:", mse_TwoTrAda_housing)

r2_score_TwoTrAda_housing = pearsonr(housing_np_test_y_list, y_pred_TwoTrAda_housing)
r2_score_TwoTrAda_housing = (r2_score_TwoTrAda_housing[0])**2
print("R^2 of TrAdaboostR2:", r2_score_TwoTrAda_housing)

print("-------------------------------------------")

Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Two-Stage TrAdaboost.R2
----------------

In [97]:
########################### AdaBoostR2 Transfer LearningL Housing #####################################################
print("Adaboost.R2 Transfer Learning")
print("-------------------------------------------")


model_Ada_housing = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 8), learning_rate=0.01, n_estimators=500) 
model_Ada_housing.fit(housing_np_train_X, housing_np_train_y_list)

y_pred_ada_housing = model_Ada_housing.predict(housing_np_test_X)
mse_Ada_housing = sqrt(mean_squared_error(housing_np_test_y, y_pred_ada_housing))
print("RMSE of Adaboost.R2:", mse_Ada_housing)

r2_score_Ada_housing = pearsonr(housing_np_test_y_list, y_pred_ada_housing)
r2_score_Ada_housing = (r2_score_Ada_housing[0])**2
print("R^2 of TrAdaboostR2:", r2_score_Ada_housing)

print("-------------------------------------------")

Adaboost.R2 Transfer Learning
-------------------------------------------
RMSE of Adaboost.R2: 7.910660690028172
R^2 of TrAdaboostR2: 0.4786809614921828
-------------------------------------------


In [98]:
########################### AdaBoostR2 Housing #####################################################
print("Adaboost.R2")
print("-------------------------------------------")


model_Ada_housing = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 8), learning_rate=0.01, n_estimators=500) 
model_Ada_housing.fit(housing_tgt_df_X, housing_tgt_df_y)

y_pred_ada_housing = model_Ada_housing.predict(housing_np_test_X)
mse_Ada_housing = sqrt(mean_squared_error(housing_np_test_y, y_pred_ada_housing))
print("RMSE of Adaboost.R2:", mse_Ada_housing)

r2_score_Ada_housing = pearsonr(housing_np_test_y_list, y_pred_ada_housing)
r2_score_Ada_housing = (r2_score_Ada_housing[0])**2
print("R^2 of TrAdaboostR2:", r2_score_Ada_housing)

print("-------------------------------------------")

Adaboost.R2
-------------------------------------------


/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


RMSE of Adaboost.R2: 8.293328796156434
R^2 of TrAdaboostR2: 0.4136328886364334
-------------------------------------------


In [99]:
######################### Kernel Mean Matching Housing #######################################
print("Kernel Mean Matching")
print("-------------------------------------------")

model_KMM_housing = KMM(get_estimator = get_estimator)
model_KMM_housing.fit(housing_np_train_X, housing_np_train_y_list, src_idx_housing, tgt_idx_housing)

y_pred_KMM_housing = model_KMM_housing.predict(housing_test_df_X) ##Using dataframe instead of the numpy matrix
mse_KMM_housing = sqrt(mean_squared_error(housing_np_test_y, y_pred_KMM_housing))
print("RMSE of KMM:", mse_KMM_housing)

r2_score_KMM_housing = pearsonr(housing_np_test_y_list, y_pred_KMM_housing)
r2_score_KMM_housing = (r2_score_KMM_housing[0])**2
print("R^2 of TrAdaboostR2:", r2_score_KMM_housing)

#model_KMM_concrete.weights_   #To get the weights of the algorithm

print("-------------------------------------------")

Kernel Mean Matching
-------------------------------------------
RMSE of KMM: 8.583829778573469
R^2 of TrAdaboostR2: 0.369751845685946
-------------------------------------------


In [ ]:
###################################### Gradient Boosting Regression Transfer Learning Housing #######################################

print("Gradient Boosting Regression Transfer Learning")
print("-------------------------------------------")


model_GBR_housing = GradientBoostingRegressor(learning_rate=0.01, max_depth=4, n_estimators=1000, subsample=0.5)
model_GBR_housing.fit(housing_np_train_X, housing_np_train_y_list)

y_pred_GBR_housing = model_GBR_housing.predict(housing_test_df_X) ##Using dataframe instead of the numpy matrix
mse_GBR_housing = sqrt(mean_squared_error(housing_np_test_y, y_pred_GBR_housing))
print("RMSE of KMM:", mse_GBR_housing)

r2_score_GBR_housing = pearsonr(housing_np_test_y_list, y_pred_GBR_housing)
r2_score_GBR_housing = (r2_score_GBR_housing[0])**2
print("R^2 of TrAdaboostR2:", r2_score_GBR_housing)

print("-------------------------------------------")

In [103]:
######################### Gradient Boosting Regression Housing #######################################

print("Gradient Boosting Regression")
print("-------------------------------------------")


model_GBR_housing = GradientBoostingRegressor(learning_rate=0.01, max_depth=4, n_estimators=1000, subsample=0.5)
model_GBR_housing.fit(housing_tgt_df_X, housing_tgt_df_y)

y_pred_GBR_housing = model_GBR_housing.predict(housing_test_df_X) ##Using dataframe instead of the numpy matrix
mse_GBR_housing = sqrt(mean_squared_error(housing_np_test_y, y_pred_GBR_housing))
print("RMSE of KMM:", mse_GBR_housing)

r2_score_GBR_housing = pearsonr(housing_np_test_y_list, y_pred_GBR_housing)
r2_score_GBR_housing = (r2_score_GBR_housing[0])**2
print("R^2 of TrAdaboostR2:", r2_score_GBR_housing)

print("-------------------------------------------")

Gradient Boosting Regression
-------------------------------------------


/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


RMSE of KMM: 7.858297178868333
R^2 of TrAdaboostR2: 0.4850629881108619
-------------------------------------------


In [104]:
################ Transformed TrAdaBoost.R2 Housing #############################################

weights_KMM = model_KMM_housing.weights_

val_mean = np.mean(weights_KMM)
weights_KMM = np.append(weights_KMM, val_mean)

housing_source_df_X_trans = housing_source_df_X.apply(lambda housing_source_df_X: housing_source_df_X * weights_KMM)
#concrete_source_df_X.multiply(weights_KMM, axis=0)

# concrete_np_train_X_trans = concrete_source_df_X_trans.to_numpy()

housing_X_df = pd.concat([housing_tgt_df_X, housing_source_df_X_trans], ignore_index=True)
housing_y_df = pd.concat([housing_tgt_df_y, housing_source_df_y], ignore_index=True)

housing_np_train_X = housing_X_df.to_numpy()
housing_np_train_y = housing_y_df.to_numpy()

housing_np_test_X = housing_test_df_X.to_numpy()
housing_np_test_y = housing_test_df_y.to_numpy()

housing_np_train_y_list = housing_np_train_y.ravel()
housing_np_test_y_list = housing_np_test_y.ravel()

# print(concrete_np_train_y_list.shape, concrete_np_train_X_trans.shape)

##################################################################
model_TwoTrAda_housing = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 100, cv=10) #, kwargs_TwoTrAda)
model_TwoTrAda_housing.fit(housing_np_train_X, housing_np_train_y_list, src_idx_housing, tgt_idx_housing)

print("Transformed Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

y_pred_TwoTrAda_housing = model_TwoTrAda_housing.predict(housing_np_test_X)

mse_TwoTrAda_housing = sqrt(mean_squared_error(housing_np_test_y, y_pred_TwoTrAda_housing))
print("RMSE of Two-Stage TrAdaboost.R2:", mse_TwoTrAda_housing)

r2_score_TwoTrAda_housing = pearsonr(housing_np_test_y_list, y_pred_TwoTrAda_housing)
r2_score_TwoTrAda_housing = (r2_score_TwoTrAda_housing[0])**2
print("R^2 of TrAdaboostR2:", r2_score_TwoTrAda_housing)

print("-------------------------------------------")

Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Transformed Two-Stage TrAdaboost.R2
-------------------------------------------
RMSE of Two-Stage TrAdaboost.R2: 8.83919577983453
R^2 of TrAdaboostR2: 0.32458544508387793
-------------------------------------------


In [105]:
#################################### STrAdaBoost.R2 Housing ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

sample_size = [len(housing_tgt_df_X), len(housing_source_df_X)]
n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

model_stradaboost_housing = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6),
                    n_estimators = n_estimators, sample_size = sample_size,
                    steps = steps, fold = fold, random_state = random_state)


model_stradaboost_housing.fit(housing_np_train_X, housing_np_train_y_list)
y_pred_stradaboost_housing = model_stradaboost_housing.predict(housing_np_test_X)

mse_stradaboost_housing = sqrt(mean_squared_error(housing_np_test_y, y_pred_stradaboost_housing))
print("RMSE of STrAdaboostR2:", mse_stradaboost_housing)

r2_score_stradaboost_housing = pearsonr(housing_np_test_y_list, y_pred_stradaboost_housing)
r2_score_stradaboost_housing = (r2_score_stradaboost_housing[0])**2
print("R^2 of STrAdaboostR2:", r2_score_stradaboost_housing)

Inside STrAdaBoost.R2
RMSE of STrAdaboostR2: 7.965035134947645
R^2 of STrAdaboostR2: 0.48541805750622513


In [100]:
######################################################## Automobile ################################################################
## horsepower column has correlation 0.4 :: [46 - 230] :: 30
#################################################################################################################################
dropcol_initial_auto = ['name']
AutoData_df = pd.read_csv('UCI_regression/MPG/Auto.csv') ## horsepower column has correlation 0.4 :: [46 - 230] :: 30
AutoData_df = AutoData_df.drop(dropcol_initial_auto, axis = 1)
print("The shape of the Input data is: ", AutoData_df.shape)

drop_col_auto = ['horsepower']

auto_tgt_df = AutoData_df.loc[(AutoData_df['horsepower'] > 80) & (AutoData_df['horsepower'] <= 110)]
auto_tgt_df = auto_tgt_df.drop(drop_col_auto, axis = 1)
auto_tgt_df = auto_tgt_df.reset_index(drop=True)
print("Target Set: ",auto_tgt_df.shape)

auto_source_df = AutoData_df.loc[(AutoData_df['horsepower'] <= 80)]
auto_source_df = auto_source_df.drop(drop_col_auto, axis = 1)
auto_source_df = auto_source_df.reset_index(drop=True)
print("Source Set: ",auto_source_df.shape)

auto_test_df = AutoData_df.loc[(AutoData_df['horsepower'] > 110)]
auto_test_df = auto_test_df.drop(drop_col_auto, axis = 1)
auto_test_df = auto_test_df.reset_index(drop=True)
print("Test Set: ",auto_test_df.shape)

#################### Splitting into features and target ####################
target_column_auto = ['mpg']

auto_tgt_df_y = auto_tgt_df[target_column_auto]
auto_tgt_df_X = auto_tgt_df.drop(target_column_auto, axis = 1)

auto_source_df_y = auto_source_df[target_column_auto]
auto_source_df_X = auto_source_df.drop(target_column_auto, axis = 1)

auto_test_df_y = auto_test_df[target_column_auto]
auto_test_df_X = auto_test_df.drop(target_column_auto, axis = 1)

############## Merging the datasets ##########################################
auto_X_df = pd.concat([auto_tgt_df_X, auto_source_df_X], ignore_index=True)
auto_y_df = pd.concat([auto_tgt_df_y, auto_source_df_y], ignore_index=True)

auto_np_train_X = auto_X_df.to_numpy()
auto_np_train_y = auto_y_df.to_numpy()

auto_np_test_X = auto_test_df_X.to_numpy()
auto_np_test_y = auto_test_df_y.to_numpy()

auto_np_train_y_list = auto_np_train_y.ravel()
auto_np_test_y_list = auto_np_test_y.ravel()

src_size_auto = len(auto_source_df_y)
tgt_size_auto = len(auto_tgt_df_y)

print("---------------------------")

The shape of the Input data is:  (392, 8)
Target Set:  (157, 7)
Source Set:  (119, 7)
Test Set:  (116, 7)
---------------------------


In [5]:
######################## Transfer Learning specifications for Auto #########################################
src_idx_auto = np.arange(start=0, stop=(src_size_auto - 1), step=1)
tgt_idx_auto = np.arange(start=src_size_auto, stop=((src_size_auto + tgt_size_auto)-1), step=1)


def get_estimator(**kwargs):
    return DecisionTreeRegressor(max_depth = 6)

kwargs_TwoTrAda = {'steps': 30,
                    'fold': 10,
                  'learning_rate': 0.1}

In [7]:
########################### TwoStagetrAdaBoostR2 Auto #######################################

model_TwoTrAda_auto = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 1000, cv=10) #, kwargs_TwoTrAda)
model_TwoTrAda_auto.fit(auto_np_train_X, auto_np_train_y_list, src_idx_auto, tgt_idx_auto)

print("Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

y_pred_TwoTrAda_auto = model_TwoTrAda_auto.predict(auto_np_test_X)
mse_TwoTrAda_auto = sqrt(mean_squared_error(auto_np_test_y, y_pred_TwoTrAda_auto))
print("RMSE of Two-Stage TrAdaboost.R2:", mse_TwoTrAda_auto)

r2_score_TwoTrAda_auto = pearsonr(auto_np_test_y_list, y_pred_TwoTrAda_auto)
r2_score_TwoTrAda_auto = (r2_score_TwoTrAda_auto[0])**2
print("R^2 of TrAdaboostR2:", r2_score_TwoTrAda_auto)

print("-------------------------------------------")

Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set val

In [8]:
########################### AdaBoostR2 Transfer LearningL Auto #####################################################
print("Adaboost.R2 Transfer Learning")
print("-------------------------------------------")


model_Ada_auto = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 8), learning_rate=0.01, n_estimators=500) 
model_Ada_auto.fit(auto_np_train_X, auto_np_train_y_list)

y_pred_ada_auto = model_Ada_auto.predict(auto_np_test_X)
mse_Ada_auto = sqrt(mean_squared_error(auto_np_test_y, y_pred_ada_auto))
print("RMSE of Adaboost.R2:", mse_Ada_auto)

r2_score_Ada_auto = pearsonr(auto_np_test_y_list, y_pred_ada_auto)
r2_score_Ada_auto = (r2_score_Ada_auto[0])**2
print("R^2 of TrAdaboostR2:", r2_score_Ada_auto)

print("-------------------------------------------")

Adaboost.R2 Transfer Learning
-------------------------------------------
RMSE of Adaboost.R2: 4.363924315238744
R^2 of TrAdaboostR2: 0.6080438774797282
-------------------------------------------


In [9]:
########################### AdaBoostR2 Auto #####################################################
print("Adaboost.R2")
print("-------------------------------------------")


model_Ada_auto = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 8), learning_rate=0.01, n_estimators=500) 
model_Ada_auto.fit(auto_tgt_df_X, auto_tgt_df_y)

y_pred_ada_auto = model_Ada_auto.predict(auto_np_test_X)
mse_Ada_auto = sqrt(mean_squared_error(auto_np_test_y, y_pred_ada_auto))
print("RMSE of Adaboost.R2:", mse_Ada_auto)

r2_score_Ada_auto = pearsonr(auto_np_test_y_list, y_pred_ada_auto)
r2_score_Ada_auto = (r2_score_Ada_auto[0])**2
print("R^2 of TrAdaboostR2:", r2_score_Ada_auto)

print("-------------------------------------------")

Adaboost.R2
-------------------------------------------


/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


RMSE of Adaboost.R2: 4.420810660355685
R^2 of TrAdaboostR2: 0.5502620404767535
-------------------------------------------


In [10]:
######################### Kernel Mean Matching Auto #######################################
print("Kernel Mean Matching")
print("-------------------------------------------")

model_KMM_auto = KMM(get_estimator = get_estimator)
model_KMM_auto.fit(auto_np_train_X, auto_np_train_y_list, src_idx_auto, tgt_idx_auto)

y_pred_KMM_auto = model_KMM_auto.predict(auto_test_df_X) ##Using dataframe instead of the numpy matrix
mse_KMM_auto = sqrt(mean_squared_error(auto_np_test_y, y_pred_KMM_auto))
print("RMSE of KMM:", mse_KMM_auto)

r2_score_KMM_auto = pearsonr(auto_np_test_y_list, y_pred_KMM_auto)
r2_score_KMM_auto = (r2_score_KMM_auto[0])**2
print("R^2 of TrAdaboostR2:", r2_score_KMM_auto)

#model_KMM_concrete.weights_   #To get the weights of the algorithm

print("-------------------------------------------")

Kernel Mean Matching
-------------------------------------------
RMSE of KMM: 4.111120791345881
R^2 of TrAdaboostR2: 0.4065209586577796
-------------------------------------------


In [12]:
######################### Gradient Boosting Regression Transfer Learning Auto #######################################

print("Gradient Boosting Regression Transfer Learning")
print("-------------------------------------------")


model_GBR_auto = GradientBoostingRegressor(learning_rate=0.01, max_depth=4, n_estimators=1000, subsample=0.5)
model_GBR_auto.fit(auto_np_train_X, auto_np_train_y_list)

y_pred_GBR_auto = model_GBR_auto.predict(auto_test_df_X) ##Using dataframe instead of the numpy matrix
mse_GBR_auto = sqrt(mean_squared_error(auto_np_test_y, y_pred_GBR_auto))
print("RMSE of KMM:", mse_GBR_auto)

r2_score_GBR_auto = pearsonr(auto_np_test_y_list, y_pred_GBR_auto)
r2_score_GBR_auto = (r2_score_GBR_auto[0])**2
print("R^2 of TrAdaboostR2:", r2_score_GBR_auto)

print("-------------------------------------------")

Gradient Boosting Regression Transfer Learning
-------------------------------------------
RMSE of KMM: 6.382657961223332
R^2 of TrAdaboostR2: 0.3405912128920165
-------------------------------------------


In [13]:
######################### Gradient Boosting Regression Auto #######################################

print("Gradient Boosting Regression")
print("-------------------------------------------")


model_GBR_auto = GradientBoostingRegressor(learning_rate=0.01, max_depth=4, n_estimators=1000, subsample=0.5)
model_GBR_auto.fit(auto_tgt_df_X, auto_tgt_df_y)

y_pred_GBR_auto = model_GBR_auto.predict(auto_test_df_X) ##Using dataframe instead of the numpy matrix
mse_GBR_auto = sqrt(mean_squared_error(auto_np_test_y, y_pred_GBR_auto))
print("RMSE of KMM:", mse_GBR_auto)

r2_score_GBR_auto = pearsonr(auto_np_test_y_list, y_pred_GBR_auto)
r2_score_GBR_auto = (r2_score_GBR_auto[0])**2
print("R^2 of TrAdaboostR2:", r2_score_GBR_auto)

print("-------------------------------------------")

Gradient Boosting Regression
-------------------------------------------


/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


RMSE of KMM: 6.200761352176353
R^2 of TrAdaboostR2: 0.3313599754308558
-------------------------------------------


In [14]:
################ Transformed TrAdaBoost.R2 Auto #############################################

weights_KMM = model_KMM_auto.weights_

val_mean = np.mean(weights_KMM)
weights_KMM = np.append(weights_KMM, val_mean)

auto_source_df_X_trans = auto_source_df_X.apply(lambda auto_source_df_X: auto_source_df_X * weights_KMM)
#concrete_source_df_X.multiply(weights_KMM, axis=0)

# concrete_np_train_X_trans = concrete_source_df_X_trans.to_numpy()

auto_X_df = pd.concat([auto_tgt_df_X, auto_source_df_X_trans], ignore_index=True)
auto_y_df = pd.concat([auto_tgt_df_y, auto_source_df_y], ignore_index=True)

auto_np_train_X = auto_X_df.to_numpy()
auto_np_train_y = auto_y_df.to_numpy()

auto_np_test_X = auto_test_df_X.to_numpy()
auto_np_test_y = auto_test_df_y.to_numpy()

auto_np_train_y_list = auto_np_train_y.ravel()
auto_np_test_y_list = auto_np_test_y.ravel()

# print(concrete_np_train_y_list.shape, concrete_np_train_X_trans.shape)

##################################################################
model_TwoTrAda_auto = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 100, cv=10) #, kwargs_TwoTrAda)
model_TwoTrAda_auto.fit(auto_np_train_X, auto_np_train_y_list, src_idx_auto, tgt_idx_auto)

print("Transformed Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

y_pred_TwoTrAda_auto = model_TwoTrAda_auto.predict(auto_np_test_X)

mse_TwoTrAda_auto = sqrt(mean_squared_error(auto_np_test_y, y_pred_TwoTrAda_auto))
print("RMSE of Two-Stage TrAdaboost.R2:", mse_Two900TrAda_auto)

r2_score_TwoTrAda_auto = pearsonr(auto_np_test_y_list, y_pred_TwoTrAda_auto)
r2_score_TwoTrAda_auto = (r2_score_TwoTrAda_auto[0])**2
print("R^2 of TrAdaboostR2:", r2_score_TwoTrAda_auto)

print("-------------------------------------------")

Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Transformed Two-Stage TrAdaboost.R2
-------------------------------------------
RMSE of Two-Stage TrAdaboost.R2: 5.235239636298763
R^2 of TrAdaboostR2: 0.37128819283084824
-------------------------------------------


In [101]:
#################################### STrAdaBoost.R2 Auto ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

sample_size = [len(auto_tgt_df_X), len(auto_source_df_X)]
n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

model_stradaboost_auto = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6),
                    n_estimators = n_estimators, sample_size = sample_size,
                    steps = steps, fold = fold, random_state = random_state)


model_stradaboost_auto.fit(auto_np_train_X, auto_np_train_y_list)
y_pred_stradaboost_auto = model_stradaboost_auto.predict(auto_np_test_X)

mse_stradaboost_auto = sqrt(mean_squared_error(auto_np_test_y, y_pred_stradaboost_auto))
print("RMSE of STrAdaboostR2:", mse_stradaboost_auto)

r2_score_stradaboost_auto = pearsonr(auto_np_test_y_list, y_pred_stradaboost_auto)
r2_score_stradaboost_auto = (r2_score_stradaboost_auto[0])**2
print("R^2 of STrAdaboostR2:", r2_score_stradaboost_auto)

Inside STrAdaBoost.R2
RMSE of STrAdaboostR2: 4.819470738317646
R^2 of STrAdaboostR2: 0.617582863408081


In [ ]:
################################################################ Ailerons ###############################################
target_ailerons = ['goal']
colnames_ailerons = ['climbRate', 'Sgz', 'p', 'q', 'curPitch', 'curRoll', 'absRoll', 'diffClb', 'diffRollRate', 'diffDiffClb', 'SeTime1',
            'SeTime2', 'SeTime3', 'SeTime4', 'SeTime5', 'SeTime6', 'SeTime7', 'SeTime8', 'SeTime9', 'SeTime10', 'SeTime11', 'SeTime12', 
            'SeTime13', 'SeTime14', 'diffSeTime1', 'diffSeTime2', 'diffSeTime3', 'diffSeTime4', 'diffSeTime5', 'diffSeTime6', 'diffSeTime7',
            'diffSeTime8', 'diffSeTime9', 'diffSeTime10', 'diffSeTime11', 'diffSeTime12', 'diffSeTime13', 'diffSeTime14', 'alpha', 'Se', 'goal']
AileronsData_df_train = pd.read_csv('UCI_regression/Ailerons/ailerons.data', header = None, names = colnames_ailerons) 
print("Ailerons Data")
# print(AileronsData_df_train.shape)
print("-------------------------------------------")

AileronsData_df_test = pd.read_csv('UCI_regression/Ailerons/ailerons.test', header = None, names = colnames_ailerons) 
# print(AileronsData_df_test.shape)

#################### Splitting with small target set and large source and test set #############
AileronsData_df_source, AileronsData_df_tgt = train_test_split(AileronsData_df_train, test_size = 0.05) ## test_size = tgt size
# print(AileronsData_df_tgt.shape, AileronsData_df_source.shape, AileronsData_df_test.shape)

AileronsData_df_train = AileronsData_df_train.reset_index(drop = True)

AileronsData_df_tgt = AileronsData_df_tgt.reset_index(drop = True)
AileronsData_df_source = AileronsData_df_source.reset_index(drop = True)
print("Target Set: ", AileronsData_df_tgt.shape)
print("Source Set: ", AileronsData_df_source.shape)
print("Test Set: ", AileronsData_df_test.shape)


AileronsData_df_test_y = AileronsData_df_test[target_ailerons]
AileronsData_df_test_X = AileronsData_df_test.drop(target_ailerons, axis = 1)

AileronsData_df_tgt_y = AileronsData_df_tgt[target_ailerons]
AileronsData_df_tgt_X = AileronsData_df_tgt.drop(target_ailerons, axis = 1)

AileronsData_df_source_y = AileronsData_df_source[target_ailerons]
AileronsData_df_source_X = AileronsData_df_source.drop(target_ailerons, axis = 1)


############## Merging the datasets #################
ailerons_X_df = pd.concat([AileronsData_df_tgt_X, AileronsData_df_source_X], ignore_index=True)
ailerons_y_df = pd.concat([AileronsData_df_tgt_y, AileronsData_df_source_y], ignore_index=True)

ailerons_np_train_X = ailerons_X_df.to_numpy()
ailerons_np_train_y = ailerons_y_df.to_numpy()

ailerons_np_test_X = AileronsData_df_test_X.to_numpy()
ailerons_np_test_y = AileronsData_df_test_y.to_numpy()

ailerons_np_train_y_list = ailerons_np_train_y.ravel()
ailerons_np_test_y_list = ailerons_np_test_y.ravel()

src_size_ailerons = len(AileronsData_df_source_y)
tgt_size_ailerons = len(AileronsData_df_tgt_y)


# AileronsData_df_train_y = AileronsData_df_train['goal']

print("-------------------------------------------")
###############################################################################################################################################

In [17]:
############# Transfer Learning specifications #########################
src_idx_ailerons = np.arange(start=0, stop=(src_size_ailerons - 1), step=1)
tgt_idx_ailerons = np.arange(start=src_size_ailerons, stop=((src_size_ailerons + tgt_size_ailerons)-1), step=1)

In [ ]:
########################### TwoStagetrAdaBoostR2 Ailerons #######################################

model_TwoTrAda_ailerons = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 1000, cv=10) #, kwargs_TwoTrAda)
model_TwoTrAda_ailerons.fit(ailerons_np_train_X, ailerons_np_train_y_list, src_idx_ailerons, tgt_idx_ailerons)

print("Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

y_pred_TwoTrAda_ailerons = model_TwoTrAda_ailerons.predict(ailerons_np_test_X)
mse_TwoTrAda_ailerons = sqrt(mean_squared_error(ailerons_np_test_y, y_pred_TwoTrAda_ailerons))
print("RMSE of Two-Stage TrAdaboost.R2:", mse_TwoTrAda_ailerons)

r2_score_TwoTrAda_ailerons = pearsonr(ailerons_np_test_y_list, y_pred_TwoTrAda_ailerons)
r2_score_TwoTrAda_ailerons = (r2_score_TwoTrAda_ailerons[0])**2
print("R^2 of TrAdaboostR2:", r2_score_TwoTrAda_ailerons)

print("-------------------------------------------")

In [18]:
########################### AdaBoostR2 Transfer LearningL Ailerons #####################################################
print("Adaboost.R2 Transfer Learning")
print("-------------------------------------------")


model_Ada_ailerons = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 8), learning_rate=0.01, n_estimators=500) 
model_Ada_ailerons.fit(ailerons_np_train_X, ailerons_np_train_y_list)

y_pred_ada_ailerons = model_Ada_ailerons.predict(ailerons_np_test_X)
mse_Ada_ailerons = sqrt(mean_squared_error(ailerons_np_test_y, y_pred_ada_ailerons))
print("RMSE of Adaboost.R2:", mse_Ada_ailerons)

r2_score_Ada_ailerons = pearsonr(ailerons_np_test_y_list, y_pred_ada_ailerons)
r2_score_Ada_ailerons = (r2_score_Ada_ailerons[0])**2
print("R^2 of TrAdaboostR2:", r2_score_Ada_ailerons)

print("-------------------------------------------")

Adaboost.R2 Transfer Learning
-------------------------------------------
RMSE of Adaboost.R2: 0.00017449887580011638
R^2 of TrAdaboostR2: 0.8129890343240704
-------------------------------------------


In [19]:
########################### AdaBoostR2 Ailerons #####################################################
print("Adaboost.R2")
print("-------------------------------------------")


model_Ada_ailerons = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 8), learning_rate=0.01, n_estimators=500) 
model_Ada_ailerons.fit(AileronsData_df_tgt_X, AileronsData_df_tgt_y)

y_pred_ada_ailerons = model_Ada_ailerons.predict(ailerons_np_test_X)
mse_Ada_ailerons = sqrt(mean_squared_error(ailerons_np_test_y, y_pred_ada_ailerons))
print("RMSE of Adaboost.R2:", mse_Ada_ailerons)

r2_score_Ada_ailerons = pearsonr(ailerons_np_test_y_list, y_pred_ada_ailerons)
r2_score_Ada_ailerons = (r2_score_Ada_ailerons[0])**2
print("R^2 of TrAdaboostR2:", r2_score_Ada_ailerons)

print("-------------------------------------------")

Adaboost.R2
-------------------------------------------


/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


RMSE of Adaboost.R2: 0.0002211541893347213
R^2 of TrAdaboostR2: 0.6992253917998658
-------------------------------------------


In [ ]:
######################### Kernel Mean Matching #######################################
print("Kernel Mean Matching")
print("-------------------------------------------")

model_KMM_ailerons = KMM(get_estimator = get_estimator)
model_KMM_ailerons.fit(ailerons_np_train_X, ailerons_np_train_y_list, src_idx_ailerons, tgt_idx_ailerons)

y_pred_KMM_ailerons = model_KMM_concrete.predict(AileronsData_df_test_X) ##Using dataframe instead of the numpy matrix
mse_KMM_ailerons = sqrt(mean_squared_error(ailerons_np_test_y, y_pred_KMM_ailerons))
print("RMSE of KMM:", mse_KMM_ailerons)

r2_score_KMM_ailerons = pearsonr(ailerons_np_test_y_list, y_pred_KMM_ailerons)
r2_score_KMM_ailerons = (r2_score_KMM_ailerons[0])**2
print("R^2 of TrAdaboostR2:", r2_score_KMM_ailerons)

#model_KMM_concrete.weights_   #To get the weights of the algorithm

print("-------------------------------------------")

In [20]:
######################### Gradient Boosting Regression Transfer Learning #######################################

print("Gradient Boosting Regression Transfer Learning")
print("-------------------------------------------")


model_GBRTL_ailerons = GradientBoostingRegressor(learning_rate=0.01, max_depth=4, n_estimators=1000, subsample=0.5)
model_GBRTL_ailerons.fit(ailerons_np_train_X, ailerons_np_train_y_list)

y_pred_GBRTL_ailerons = model_GBRTL_ailerons.predict(AileronsData_df_test_X) ##Using dataframe instead of the numpy matrix
mse_GBRTL_ailerons = sqrt(mean_squared_error(ailerons_np_test_y, y_pred_GBRTL_ailerons))
print("RMSE of KMM:", mse_GBRTL_ailerons)

r2_score_GBRTL_ailerons = pearsonr(ailerons_np_test_y_list, y_pred_GBRTL_ailerons)
r2_score_GBRTL_ailerons = (r2_score_GBRTL_ailerons[0])**2
print("R^2 of TrAdaboostR2:", r2_score_GBRTL_ailerons)

print("-------------------------------------------")

Gradient Boosting Regression Transfer Learning
-------------------------------------------
RMSE of KMM: 0.00016086118104770237
R^2 of TrAdaboostR2: 0.8410246331188738
-------------------------------------------


In [21]:
######################### Gradient Boosting Regression #######################################

print("Gradient Boosting Regression")
print("-------------------------------------------")


model_GBR_ailerons = GradientBoostingRegressor(learning_rate=0.01, max_depth=4, n_estimators=1000, subsample=0.5)
model_GBR_ailerons.fit(AileronsData_df_tgt_X, AileronsData_df_tgt_y)

y_pred_GBR_ailerons = model_GBR_ailerons.predict(AileronsData_df_test_X) ##Using dataframe instead of the numpy matrix
mse_GBR_ailerons = sqrt(mean_squared_error(ailerons_np_test_y, y_pred_GBR_ailerons))
print("RMSE of KMM:", mse_GBR_ailerons)

r2_score_GBR_ailerons = pearsonr(ailerons_np_test_y_list, y_pred_GBR_ailerons)
r2_score_GBR_ailerons = (r2_score_GBR_ailerons[0])**2
print("R^2 of TrAdaboostR2:", r2_score_GBR_ailerons)

print("-------------------------------------------")

Gradient Boosting Regression
-------------------------------------------


/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


RMSE of KMM: 0.00019240831285250804
R^2 of TrAdaboostR2: 0.7720629536034364
-------------------------------------------


In [22]:
################ Transformed TrAdaBoost.R2 Ailerons #############################################

weights_KMM = model_KMM_ailerons.weights_

val_mean = np.mean(weights_KMM)
weights_KMM = np.append(weights_KMM, val_mean)

ailerons_source_df_X_trans = ailerons_source_df_X.apply(lambda ailerons_source_df_X: ailerons_source_df_X * weights_KMM)
#concrete_source_df_X.multiply(weights_KMM, axis=0)

# concrete_np_train_X_trans = concrete_source_df_X_trans.to_numpy()

ailerons_X_df = pd.concat([ailerons_tgt_df_X, ailerons_source_df_X_trans], ignore_index=True)
ailerons_y_df = pd.concat([ailerons_tgt_df_y, ailerons_source_df_y], ignore_index=True)

ailerons_np_train_X = ailerons_X_df.to_numpy()
ailerons_np_train_y = ailerons_y_df.to_numpy()

ailerons_np_test_X = ailerons_test_df_X.to_numpy()
ailerons_np_test_y = ailerons_test_df_y.to_numpy()

ailerons_np_train_y_list = ailerons_np_train_y.ravel()
ailerons_np_test_y_list = ailerons_np_test_y.ravel()

# print(concrete_np_train_y_list.shape, concrete_np_train_X_trans.shape)

##################################################################
model_TwoTrAda_ailerons = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 100, cv=10) #, kwargs_TwoTrAda)
model_TwoTrAda_ailerons.fit(ailerons_np_train_X, ailerons_np_train_y_list, src_idx_ailerons, tgt_idx_ailerons)

print("Transformed Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

y_pred_TwoTrAda_ailerons = model_TwoTrAda_ailerons.predict(ailerons_np_test_X)

mse_TwoTrAda_ailerons = sqrt(mean_squared_error(ailerons_np_test_y, y_pred_TwoTrAda_ailerons))
print("RMSE of Two-Stage TrAdaboost.R2:", mse_TwoTrAda_ailerons)

r2_score_TwoTrAda_ailerons = pearsonr(ailerons_np_test_y_list, y_pred_TwoTrAda_ailerons)
r2_score_TwoTrAda_ailerons = (r2_score_TwoTrAda_ailerons[0])**2
print("R^2 of TrAdaboostR2:", r2_score_TwoTrAda_ailerons)

print("-------------------------------------------")

NameError: name 'model_KMM_ailerons' is not defined

In [26]:
#################################### STrAdaBoost.R2 Ailerons ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

sample_size = [len(AileronsData_df_tgt_X), len(AileronsData_df_source_X)]
n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

model_stradaboost_ailerons = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6),
                    n_estimators = n_estimators, sample_size = sample_size,
                    steps = steps, fold = fold, random_state = random_state)


model_stradaboost_ailerons.fit(ailerons_np_train_X, ailerons_np_train_y_list)
y_pred_stradaboost_ailerons = model_stradaboost_ailerons.predict(ailerons_np_test_X)

mse_stradaboost_ailerons = sqrt(mean_squared_error(ailerons_np_test_y, y_pred_stradaboost_ailerons))
print("RMSE of STrAdaboostR2:", mse_stradaboost_ailerons)

r2_score_stradaboost_ailerons = pearsonr(ailerons_np_test_y_list, y_pred_stradaboost_ailerons)
r2_score_stradaboost_ailerons = (r2_score_stradaboost_ailerons[0])**2
print("R^2 of STrAdaboostR2:", r2_score_stradaboost_ailerons)

Inside STrAdaBoost.R2
RMSE of STrAdaboostR2: 0.00017697035377550353
R^2 of STrAdaboostR2: 0.8151835442224004


In [102]:
################################### Elevators ###########################################
target_elevators = ['Goal']
colnames_elevators = ['climbRate', 'Sgz', 'p', 'q', 'curRoll', 'absRoll', 'diffClb', 'diffRollRate', 'diffDiffClb', 'SaTime1', 'SaTime2', 
                      'SaTime3', 'SaTime4', 'diffSaTime1', 'diffSaTime2', 'diffSaTime3', 'diffSaTime4', 'Sa', 'Goal']
Elevators_df_train = pd.read_csv('UCI_regression/Elevators/elevators.data', header = None, names = colnames_elevators)
print("Elevators Data")
print(Elevators_df_train.shape)

Elevators_df_test = pd.read_csv('UCI_regression/Elevators/elevators.test', header = None, names = colnames_elevators)
print(Elevators_df_test.shape)

#################### Splitting with small target set and large source and test set #############
Elevators_df_source, Elevators_df_tgt = train_test_split(Elevators_df_train, test_size = 0.05) ## test_size = tgt size
# print(Elevators_df_tgt.shape, Elevators_df_source.shape, Elevators_df_test.shape)

Elevators_df_tgt = Elevators_df_tgt.reset_index(drop = True)
Elevators_df_source = Elevators_df_source.reset_index(drop = True)
print("Target Set: ", Elevators_df_tgt.shape)
print("Source Set: ", Elevators_df_source.shape)
print("Test Set: ", Elevators_df_test.shape)


Elevators_df_test_y = Elevators_df_test[target_elevators]
Elevators_df_test_X = Elevators_df_test.drop(target_elevators, axis = 1)

Elevators_df_tgt_y = Elevators_df_tgt[target_elevators]
Elevators_df_tgt_X = Elevators_df_tgt.drop(target_elevators, axis = 1)

Elevators_df_source_y = Elevators_df_source[target_elevators]
Elevators_df_source_X = Elevators_df_source.drop(target_elevators, axis = 1)


############## Merging the datasets #################
elevators_X_df = pd.concat([Elevators_df_tgt_X, Elevators_df_source_X], ignore_index=True)
elevators_y_df = pd.concat([Elevators_df_tgt_y, Elevators_df_source_y], ignore_index=True)

elevators_np_train_X = elevators_X_df.to_numpy()
elevators_np_train_y = elevators_y_df.to_numpy()

elevators_np_test_X = Elevators_df_test_X.to_numpy()
elevators_np_test_y = Elevators_df_test_y.to_numpy()

elevators_np_train_y_list = elevators_np_train_y.ravel()
elevators_np_test_y_list = elevators_np_test_y.ravel()

src_size_elevators = len(Elevators_df_source_y)
tgt_size_elevators = len(AileronsData_df_tgt_y)

print("---------------------------")

###############################################################################################################################################

Elevators Data
(8752, 19)
(7847, 19)
Target Set:  (438, 19)
Source Set:  (8314, 19)
Test Set:  (7847, 19)
---------------------------


In [35]:
############# Transfer Learning specifications #########################
src_idx_elevators = np.arange(start=0, stop=(src_size_elevators - 1), step=1)
tgt_idx_elevators = np.arange(start=src_size_elevators, stop=((src_size_elevators + tgt_size_elevators)-1), step=1)

In [ ]:
########################### TwoStagetrAdaBoostR2 Elevators #######################################

model_TwoTrAda_elevators = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 1000, cv=10) #, kwargs_TwoTrAda)
model_TwoTrAda_elevators.fit(elevators_np_train_X, elevators_np_train_y_list, src_idx_elevators, tgt_idx_elevators)

print("Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

y_pred_TwoTrAda_elevators = model_TwoTrAda_elevators.predict(elevators_np_test_X)
mse_TwoTrAda_elevators = sqrt(mean_squared_error(elevators_np_test_y, y_pred_TwoTrAda_elevators))
print("RMSE of Two-Stage TrAdaboost.R2:", mse_TwoTrAda_elevators)

r2_score_TwoTrAda_elevators = pearsonr(elevators_np_test_y_list, y_pred_TwoTrAda_elevators)
r2_score_TwoTrAda_elevators = (r2_score_TwoTrAda_elevators[0])**2
print("R^2 of TrAdaboostR2:", r2_score_TwoTrAda_elevators)

print("-------------------------------------------")

In [38]:
########################### AdaBoostR2 Transfer Learning Elevators #####################################################
print("Adaboost.R2 Transfer Learning")
print("-------------------------------------------")

model_Ada_elevators = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 8), learning_rate=0.01, n_estimators=500) 
model_Ada_elevators.fit(elevators_np_train_X, elevators_np_train_y_list)

y_pred_ada_elevators = model_Ada_elevators.predict(elevators_np_test_X)
mse_Ada_elevators = sqrt(mean_squared_error(elevators_np_test_y, y_pred_ada_elevators))
print("RMSE of Adaboost.R2:", mse_Ada_elevators)

r2_score_Ada_elevators = pearsonr(elevators_np_test_y_list, y_pred_ada_elevators)
r2_score_Ada_elevators = (r2_score_Ada_elevators[0])**2
print("R^2 of TrAdaboostR2:", r2_score_Ada_elevators)

print("-------------------------------------------")

Adaboost.R2 Transfer Learning
-------------------------------------------
RMSE of Adaboost.R2: 0.0033531239365182726
R^2 of TrAdaboostR2: 0.7310570733716446
-------------------------------------------


In [44]:
########################### AdaBoostR2 Elevators #####################################################
print("Adaboost.R2")
print("-------------------------------------------")


model_Ada_elevators = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 8), learning_rate=0.01, n_estimators=500) 
model_Ada_elevators.fit(elevators_np_train_X, elevators_np_train_y_list)

y_pred_ada_elevators = model_Ada_elevators.predict(elevators_np_test_X)
mse_Ada_elevators = sqrt(mean_squared_error(elevators_np_test_y, y_pred_ada_elevators))
print("RMSE of Adaboost.R2:", mse_Ada_elevators)

r2_score_Ada_elevators = pearsonr(elevators_np_test_y_list, y_pred_ada_elevators)
r2_score_Ada_elevators = (r2_score_Ada_elevators[0])**2
print("R^2 of TrAdaboostR2:", r2_score_Ada_elevators)

print("-------------------------------------------")

Adaboost.R2
-------------------------------------------
RMSE of Adaboost.R2: 0.0033468167448132517
R^2 of TrAdaboostR2: 0.7322875513444026
-------------------------------------------


In [45]:
######################### Kernel Mean Matching Elevators #######################################
print("Kernel Mean Matching")
print("-------------------------------------------")

model_KMM_elevators = KMM(get_estimator = get_estimator)
model_KMM_elevators.fit(elevators_np_train_X, elevators_np_train_y_list, src_idx_elevators, tgt_idx_elevators)

y_pred_KMM_elevators = model_KMM_elevators.predict(elevators_df_test_X) ##Using dataframe instead of the numpy matrix
mse_KMM_elevators = sqrt(mean_squared_error(elevators_np_test_y, y_pred_KMM_elevators))
print("RMSE of KMM:", mse_KMM_elevators)

r2_score_KMM_elevators = pearsonr(elevators_np_test_y_list, y_pred_KMM_elevators)
r2_score_KMM_elevators = (r2_score_KMM_elevators[0])**2
print("R^2 of TrAdaboostR2:", r2_score_KMM_elevators)

#model_KMM_concrete.weights_   #To get the weights of the algorithm

print("-------------------------------------------")

Kernel Mean Matching
-------------------------------------------


KeyboardInterrupt: 

In [47]:
######################### Gradient Boosting Regression Transfer Learning Elevators #######################################

print("Gradient Boosting Regression Transfer Learning")
print("-------------------------------------------")


model_GBRTL_elevators = GradientBoostingRegressor(learning_rate=0.01, max_depth=4, n_estimators=1000, subsample=0.5)
model_GBRTL_elevators.fit(elevators_np_train_X, elevators_np_train_y_list)

y_pred_GBRTL_elevators = model_GBRTL_elevators.predict(Elevators_df_test_X) ##Using dataframe instead of the numpy matrix
mse_GBRTL_elevators = sqrt(mean_squared_error(elevators_np_test_y, y_pred_GBRTL_elevators))
print("RMSE of KMM:", mse_GBRTL_elevators)

r2_score_GBRTL_elevators = pearsonr(elevators_np_test_y_list, y_pred_GBRTL_elevators)
r2_score_GBRTL_elevators = (r2_score_GBRTL_elevators[0])**2
print("R^2 of TrAdaboostR2:", r2_score_GBRTL_elevators)

print("-------------------------------------------")

Gradient Boosting Regression Transfer Learning
-------------------------------------------
RMSE of KMM: 0.00290000061975912
R^2 of TrAdaboostR2: 0.7993369840871682
-------------------------------------------


In [49]:
######################### Gradient Boosting Regression Elevators #######################################

print("Gradient Boosting Regression")
print("-------------------------------------------")


model_GBR_elevators = GradientBoostingRegressor(learning_rate=0.01, max_depth=4, n_estimators=1000, subsample=0.5)
model_GBR_elevators.fit(Elevators_df_tgt_X, Elevators_df_tgt_y)

y_pred_GBR_elevators = model_GBR_elevators.predict(Elevators_df_test_X) ##Using dataframe instead of the numpy matrix
mse_GBR_elevators = sqrt(mean_squared_error(elevators_np_test_y, y_pred_GBR_elevators))
print("RMSE of KMM:", mse_GBR_elevators)

r2_score_GBR_elevators = pearsonr(elevators_np_test_y_list, y_pred_GBR_elevators)
r2_score_GBR_elevators = (r2_score_GBR_elevators[0])**2
print("R^2 of TrAdaboostR2:", r2_score_GBR_elevators)

print("-------------------------------------------")

Gradient Boosting Regression
-------------------------------------------


/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


RMSE of KMM: 0.004249983595276213
R^2 of TrAdaboostR2: 0.5688668075278818
-------------------------------------------


In [50]:
################ Transformed TrAdaBoost.R2 Elevators #############################################

weights_KMM = model_KMM_elevators.weights_

val_mean = np.mean(weights_KMM)
weights_KMM = np.append(weights_KMM, val_mean)

elevators_source_df_X_trans = Elevators_source_df_X.apply(lambda Elevators_source_df_X: Elevators_source_df_X * weights_KMM)
#concrete_source_df_X.multiply(weights_KMM, axis=0)

# concrete_np_train_X_trans = concrete_source_df_X_trans.to_numpy()

elevators_X_df = pd.concat([elevators_tgt_df_X, elevators_source_df_X_trans], ignore_index=True)
elevators_y_df = pd.concat([elevators_tgt_df_y, elevators_source_df_y], ignore_index=True)

elevators_np_train_X = elevators_X_df.to_numpy()
elevators_np_train_y = elevators_y_df.to_numpy()

elevators_np_test_X = elevators_test_df_X.to_numpy()
elevators_np_test_y = elevators_test_df_y.to_numpy()

elevators_np_train_y_list = elevators_np_train_y.ravel()
elevators_np_test_y_list = elevators_np_test_y.ravel()

# print(concrete_np_train_y_list.shape, concrete_np_train_X_trans.shape)

##################################################################
model_TwoTrAda_elevators = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 100, cv=10) #, kwargs_TwoTrAda)
model_TwoTrAda_elevators.fit(elevators_np_train_X, elevators_np_train_y_list, src_idx_elevators, tgt_idx_elevators)

print("Transformed Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

y_pred_TwoTrAda_elevators = model_TwoTrAda_elevators.predict(elevators_np_test_X)

mse_TwoTrAda_elevators = sqrt(mean_squared_error(elevators_np_test_y, y_pred_TwoTrAda_elevators))
print("RMSE of Two-Stage TrAdaboost.R2:", mse_TwoTrAda_elevators)

r2_score_TwoTrAda_elevators = pearsonr(elevators_np_test_y_list, y_pred_TwoTrAda_elevators)
r2_score_TwoTrAda_elevators = (r2_score_TwoTrAda_elevators[0])**2
print("R^2 of TrAdaboostR2:", r2_score_TwoTrAda_elevators)

print("-------------------------------------------")

AttributeError: 'KMM' object has no attribute 'weights_'

In [103]:
#################################### STrAdaBoost.R2 Elevators ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

sample_size = [len(Elevators_df_tgt_X), len(Elevators_df_source_X)]
n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

model_stradaboost_elevators = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6),
                    n_estimators = n_estimators, sample_size = sample_size,
                    steps = steps, fold = fold, random_state = random_state)


model_stradaboost_elevators.fit(elevators_np_train_X, elevators_np_train_y_list)
y_pred_stradaboost_elevators = model_stradaboost_elevators.predict(elevators_np_test_X)

mse_stradaboost_elevators = sqrt(mean_squared_error(elevators_np_test_y, y_pred_stradaboost_elevators))
print("RMSE of STrAdaboostR2:", mse_stradaboost_elevators)

r2_score_stradaboost_elevators = pearsonr(elevators_np_test_y_list, y_pred_stradaboost_elevators)
r2_score_stradaboost_elevators = (r2_score_stradaboost_elevators[0])**2
print("R^2 of STrAdaboostR2:", r2_score_stradaboost_elevators)

Inside STrAdaBoost.R2
RMSE of STrAdaboostR2: 0.0038173854763371958
R^2 of STrAdaboostR2: 0.774423111177669


In [104]:
############################ PoleTelecomm ####################################################################
target_poleTelecomm = ['foo']
colnames_poleTelecomm = ['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20',
                        'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39',
                        'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'foo']
poleTelecomm_df_train = pd.read_csv('UCI_regression/PoleTelecomm/pol.data', header = None, names = colnames_poleTelecomm)
print("Pole Telecomm Data")
print(poleTelecomm_df_train.shape)

poleTelecomm_df_test = pd.read_csv('UCI_regression/PoleTelecomm/pol.test', header = None, names = colnames_poleTelecomm)
print(poleTelecomm_df_test.shape)

#################### Splitting with small target set and large source and test set #############
poleTelecomm_df_source, poleTelecomm_df_tgt = train_test_split(poleTelecomm_df_train, test_size = 0.05) ## test_size = tgt size

poleTelecomm_df_tgt = poleTelecomm_df_tgt.reset_index(drop = True)
poleTelecomm_df_source = poleTelecomm_df_source.reset_index(drop = True)
print("Target Set: ", poleTelecomm_df_tgt.shape)
print("Source Set: ", poleTelecomm_df_source.shape)
print("Test Set: ", poleTelecomm_df_test.shape)


poleTelecomm_df_test_y = poleTelecomm_df_test[target_poleTelecomm]
poleTelecomm_df_test_X = poleTelecomm_df_test.drop(target_poleTelecomm, axis = 1)

poleTelecomm_df_tgt_y = poleTelecomm_df_tgt[target_poleTelecomm]
poleTelecomm_df_tgt_X = poleTelecomm_df_tgt.drop(target_poleTelecomm, axis = 1)

poleTelecomm_df_source_y = poleTelecomm_df_source[target_poleTelecomm]
poleTelecomm_df_source_X = poleTelecomm_df_source.drop(target_poleTelecomm, axis = 1)

############## Merging the datasets #################
poleTelecomm_X_df = pd.concat([poleTelecomm_df_tgt_X, poleTelecomm_df_source_X], ignore_index=True)
poleTelecomm_y_df = pd.concat([poleTelecomm_df_tgt_y, poleTelecomm_df_source_y], ignore_index=True)

poleTelecomm_np_train_X = poleTelecomm_X_df.to_numpy()
poleTelecomm_np_train_y = poleTelecomm_y_df.to_numpy()

poleTelecomm_np_test_X = poleTelecomm_df_test_X.to_numpy()
poleTelecomm_np_test_y = poleTelecomm_df_test_y.to_numpy()

poleTelecomm_np_train_y_list = poleTelecomm_np_train_y.ravel()
poleTelecomm_np_test_y_list = poleTelecomm_np_test_y.ravel()

src_size_poleTelecomm = len(poleTelecomm_df_source_y)
tgt_size_poleTelecomm = len(poleTelecomm_df_tgt_y)

print("---------------------------")

Pole Telecomm Data
(5000, 49)
(10000, 49)
Target Set:  (250, 49)
Source Set:  (4750, 49)
Test Set:  (10000, 49)
---------------------------


In [ ]:
################################## Transfer Learning specifications #########################################################
src_idx_poleTelecomm = np.arange(start=0, stop=(src_size_poleTelecomm - 1), step=1)
tgt_idx_poleTelecomm = np.arange(start=src_size_poleTelecomm, stop=((src_size_poleTelecomm + tgt_size_poleTelecomm)-1), step=1)

In [ ]:
########################### TwoStagetrAdaBoostR2 PoleTelecomm #######################################

model_TwoTrAda_poleTelecomm = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 1000, cv=10) #, kwargs_TwoTrAda)
model_TwoTrAda_poleTelecomm.fit(poleTelecomm_np_train_X, poleTelecomm_np_train_y_list, src_idx_poleTelecomm, tgt_idx_poleTelecomm)

print("Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

y_pred_TwoTrAda_poleTelecomm = model_TwoTrAda_poleTelecomm.predict(poleTelecomm_np_test_X)
mse_TwoTrAda_poleTelecomm = sqrt(mean_squared_error(poleTelecomm_np_test_y, y_pred_TwoTrAda_poleTelecomm))
print("RMSE of Two-Stage TrAdaboost.R2:", mse_TwoTrAda_poleTelecomm)

r2_score_TwoTrAda_poleTelecomm = pearsonr(poleTelecomm_np_test_y_list, y_pred_TwoTrAda_poleTelecomm)
r2_score_TwoTrAda_poleTelecomm = (r2_score_TwoTrAda_poleTelecomm[0])**2
print("R^2 of TrAdaboostR2:", r2_score_TwoTrAda_poleTelecomm)

print("-------------------------------------------")

In [55]:
########################### AdaBoostR2 Transfer Learning PoleTelecomm #####################################################
print("Adaboost.R2 Transfer Learning")
print("-------------------------------------------")

model_Ada_poleTelecomm = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 8), learning_rate=0.01, n_estimators=500) 
model_Ada_poleTelecomm.fit(poleTelecomm_np_train_X, poleTelecomm_np_train_y_list)

y_pred_ada_poleTelecomm = model_Ada_poleTelecomm.predict(poleTelecomm_np_test_X)
mse_Ada_poleTelecomm = sqrt(mean_squared_error(poleTelecomm_np_test_y, y_pred_ada_poleTelecomm))
print("RMSE of Adaboost.R2:", mse_Ada_poleTelecomm)

r2_score_Ada_poleTelecomm = pearsonr(poleTelecomm_np_test_y_list, y_pred_ada_poleTelecomm)
r2_score_Ada_poleTelecomm = (r2_score_Ada_poleTelecomm[0])**2
print("R^2 of TrAdaboostR2:", r2_score_Ada_poleTelecomm)

print("-------------------------------------------")

Adaboost.R2 Transfer Learning
-------------------------------------------
RMSE of Adaboost.R2: 7.215295917108106
R^2 of TrAdaboostR2: 0.9710338730753431
-------------------------------------------


In [56]:
########################### AdaBoostR2 PoleTelecomm #####################################################
print("Adaboost.R2")
print("-------------------------------------------")


model_Ada_poleTelecomm = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 8), learning_rate=0.01, n_estimators=500) 
model_Ada_poleTelecomm.fit(poleTelecomm_np_train_X, poleTelecomm_np_train_y_list)

y_pred_ada_poleTelecomm = model_Ada_poleTelecomm.predict(poleTelecomm_np_test_X)
mse_Ada_poleTelecomm = sqrt(mean_squared_error(poleTelecomm_np_test_y, y_pred_ada_poleTelecomm))
print("RMSE of Adaboost.R2:", mse_Ada_poleTelecomm)

r2_score_Ada_poleTelecomm = pearsonr(poleTelecomm_np_test_y_list, y_pred_ada_poleTelecomm)
r2_score_Ada_poleTelecomm = (r2_score_Ada_poleTelecomm[0])**2
print("R^2 of TrAdaboostR2:", r2_score_Ada_poleTelecomm)

print("-------------------------------------------")

Adaboost.R2
-------------------------------------------
RMSE of Adaboost.R2: 7.227106916829483
R^2 of TrAdaboostR2: 0.9709137106368763
-------------------------------------------


In [ ]:
######################### Kernel Mean Matching PoleTelecomm #######################################
print("Kernel Mean Matching")
print("-------------------------------------------")

model_KMM_poleTelecomm = KMM(get_estimator = get_estimator)
model_KMM_poleTelecomm.fit(poleTelecomm_np_train_X, poleTelecomm_np_train_y_list, src_idx_poleTelecomm, tgt_idx_poleTelecomm)

y_pred_KMM_poleTelecomm = model_KMM_elevators.predict(poleTelecomm_df_test_X) ##Using dataframe instead of the numpy matrix
mse_KMM_poleTelecomm = sqrt(mean_squared_error(poleTelecomm_np_test_y, y_pred_KMM_poleTelecomm))
print("RMSE of KMM:", mse_KMM_poleTelecomm)

r2_score_KMM_poleTelecomm = pearsonr(poleTelecomm_np_test_y_list, y_pred_KMM_poleTelecomm)
r2_score_KMM_poleTelecomm = (r2_score_KMM_poleTelecomm[0])**2
print("R^2 of TrAdaboostR2:", r2_score_KMM_poleTelecomm)

#model_KMM_concrete.weights_   #To get the weights of the algorithm

print("-------------------------------------------")

In [57]:
######################### Gradient Boosting Regression Transfer Learning PoleTelecomm #######################################

print("Gradient Boosting Regression Transfer Learning")
print("-------------------------------------------")


model_GBRTL_poleTelecomm = GradientBoostingRegressor(learning_rate=0.01, max_depth=4, n_estimators=1000, subsample=0.5)
model_GBRTL_poleTelecomm.fit(poleTelecomm_np_train_X, poleTelecomm_np_train_y_list)

y_pred_GBRTL_poleTelecomm = model_GBRTL_poleTelecomm.predict(poleTelecomm_df_test_X) ##Using dataframe instead of the numpy matrix
mse_GBRTL_poleTelecomm = sqrt(mean_squared_error(poleTelecomm_np_test_y, y_pred_GBRTL_poleTelecomm))
print("RMSE of KMM:", mse_GBRTL_poleTelecomm)

r2_score_GBRTL_poleTelecomm = pearsonr(poleTelecomm_np_test_y_list, y_pred_GBRTL_poleTelecomm)
r2_score_GBRTL_poleTelecomm = (r2_score_GBRTL_poleTelecomm[0])**2
print("R^2 of TrAdaboostR2:", r2_score_GBRTL_poleTelecomm)

print("-------------------------------------------")

Gradient Boosting Regression Transfer Learning
-------------------------------------------
RMSE of KMM: 7.886908269088648
R^2 of TrAdaboostR2: 0.9645003396543416
-------------------------------------------


In [106]:
######################### Gradient Boosting Regression PoleTelecomm #######################################

print("Gradient Boosting Regression")
print("-------------------------------------------")


model_GBR_poleTelecomm = GradientBoostingRegressor(learning_rate=0.01, max_depth=4, n_estimators=1000, subsample=0.5)
model_GBR_poleTelecomm.fit(poleTelecomm_df_tgt_X, poleTelecomm_df_tgt_y)

y_pred_GBR_poleTelecomm = model_GBR_poleTelecomm.predict(poleTelecomm_df_test_X) ##Using dataframe instead of the numpy matrix
mse_GBR_poleTelecomm = sqrt(mean_squared_error(poleTelecomm_np_test_y, y_pred_GBR_poleTelecomm))
print("RMSE of KMM:", mse_GBR_poleTelecomm)

r2_score_GBR_poleTelecomm = pearsonr(poleTelecomm_np_test_y_list, y_pred_GBR_poleTelecomm)
r2_score_GBR_poleTelecomm = (r2_score_GBR_poleTelecomm[0])**2
print("R^2 of TrAdaboostR2:", r2_score_GBR_poleTelecomm)

print("-------------------------------------------")

Gradient Boosting Regression
-------------------------------------------


/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


RMSE of KMM: 14.44710686981929
R^2 of TrAdaboostR2: 0.8804730765202913
-------------------------------------------


In [ ]:
################ Transformed TrAdaBoost.R2 PoleTelecomm #############################################

weights_KMM = model_KMM_poleTelecomm.weights_

val_mean = np.mean(weights_KMM)
weights_KMM = np.append(weights_KMM, val_mean)

poleTelecomm_source_df_X_trans = poleTelecomm_source_df_X.apply(lambda poleTelecomm_source_df_X: poleTelecomm_source_df_X * weights_KMM)
#concrete_source_df_X.multiply(weights_KMM, axis=0)

# concrete_np_train_X_trans = concrete_source_df_X_trans.to_numpy()

poleTelecomm_X_df = pd.concat([poleTelecomm_tgt_df_X, poleTelecomm_source_df_X_trans], ignore_index=True)
poleTelecomm_y_df = pd.concat([poleTelecomm_tgt_df_y, poleTelecomm_source_df_y], ignore_index=True)

poleTelecomm_np_train_X = poleTelecomm_X_df.to_numpy()
poleTelecomm_np_train_y = poleTelecomm_y_df.to_numpy()

poleTelecomm_np_test_X = poleTelecomm_test_df_X.to_numpy()
poleTelecomm_np_test_y = poleTelecomm_test_df_y.to_numpy()

poleTelecomm_np_train_y_list = poleTelecomm_np_train_y.ravel()
poleTelecomm_np_test_y_list = poleTelecomm_np_test_y.ravel()

# print(concrete_np_train_y_list.shape, concrete_np_train_X_trans.shape)

##################################################################
model_TwoTrAda_poleTelecomm = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 100, cv=10) #, kwargs_TwoTrAda)
model_TwoTrAda_poleTelecomm.fit(poleTelecomm_np_train_X, poleTelecomm_np_train_y_list, src_idx_poleTelecomm, tgt_idx_poleTelecomm)

print("Transformed Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

y_pred_TwoTrAda_poleTelecomm = model_TwoTrAda_poleTelecomm.predict(poleTelecomm_np_test_X)

mse_TwoTrAda_poleTelecomm = sqrt(mean_squared_error(poleTelecomm_np_test_y, y_pred_TwoTrAda_poleTelecomm))
print("RMSE of Two-Stage TrAdaboost.R2:", mse_TwoTrAda_poleTelecomm)

r2_score_TwoTrAda_poleTelecomm = pearsonr(poleTelecomm_np_test_y_list, y_pred_TwoTrAda_poleTelecomm)
r2_score_TwoTrAda_poleTelecomm = (r2_score_TwoTrAda_poleTelecomm[0])**2
print("R^2 of TrAdaboostR2:", r2_score_TwoTrAda_poleTelecomm)

print("-------------------------------------------")

In [59]:
#################################### STrAdaBoost.R2 PoleTelecomm ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

sample_size = [len(poleTelecomm_df_tgt_X), len(poleTelecomm_df_source_X)]
n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

model_stradaboost_poleTelecomm = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6),
                    n_estimators = n_estimators, sample_size = sample_size,
                    steps = steps, fold = fold, random_state = random_state)


model_stradaboost_poleTelecomm.fit(poleTelecomm_np_train_X, poleTelecomm_np_train_y_list)
y_pred_stradaboost_poleTelecomm = model_stradaboost_poleTelecomm.predict(poleTelecomm_np_test_X)

mse_stradaboost_poleTelecomm = sqrt(mean_squared_error(poleTelecomm_np_test_y, y_pred_stradaboost_poleTelecomm))
print("RMSE of STrAdaboostR2:", mse_stradaboost_poleTelecomm)

r2_score_stradaboost_poleTelecomm = pearsonr(poleTelecomm_np_test_y_list, y_pred_stradaboost_poleTelecomm)
r2_score_stradaboost_poleTelecomm = (r2_score_stradaboost_poleTelecomm[0])**2
print("R^2 of STrAdaboostR2:", r2_score_stradaboost_poleTelecomm)

Inside STrAdaBoost.R2
RMSE of STrAdaboostR2: 11.70628467573206
R^2 of STrAdaboostR2: 0.9640301238839806


In [8]:
################################### Abalone ###########################################
#### range: [0.0 - 1.130]
#### Mid of correlation variable: Whole_weight
#### [0, 0.12] [0.12, 0.15], [0.15, 1.130]
#### Target variable: Rings
#######################################################################################
target_var_abalone = ['Rings']
colnames_abalone = ['Sex', 'Length', 'Diameter', 'Height', 'Whole_weight', 'Shucked_weight', 'Viscera_weight', 'Shell_weight', 'Rings']
AbaloneData_df = pd.read_csv('UCI_regression/Abalone/abalone.data', header = None, names = colnames_abalone)

gender = {'M': 1,'F': 2, 'I': 3} 
AbaloneData_df.Sex = [gender[item] for item in AbaloneData_df.Sex] 

print("Abalone Data")
print(AbaloneData_df.shape)

########## Corr Abalone ################
# print("The correlation matrix is: ")
# print(AbaloneData_df.corr()['Rings'])

##################### Splitting in 3 equal parts #######################################
# print(AbaloneData_df.sort_values('Whole_weight')['Whole_weight'])
# print(len(AbaloneData_df.loc[(AbaloneData_df['Whole_weight'] <= 0.5)]))
# print(len(AbaloneData_df.loc[(AbaloneData_df['Whole_weight'] > 0.5) & (AbaloneData_df['Whole_weight'] <= 1.0)]))
# print(len(AbaloneData_df.loc[(AbaloneData_df['Whole_weight'] > 1.0)]))

#################### Splitting with small target set and large source and test set #############
# AbaloneData_df_remain, AbaloneData_df_tgt = train_test_split(AbaloneData_df, test_size = 0.05) ## test_size = tgt size
# AbaloneData_df_source, AbaloneData_df_test = train_test_split(AbaloneData_df_remain, test_size = 0.3) ## test_size = tgt size
# print(AbaloneData_df_tgt.shape, AbaloneData_df_source.shape, AbaloneData_df_test.shape)


drop_col_abalone = ['Whole_weight']

abalone_tgt_df = AbaloneData_df.loc[(AbaloneData_df['Whole_weight'] <= 0.5)]
abalone_tgt_df = abalone_tgt_df.drop(drop_col_abalone, axis = 1)
abalone_tgt_df = abalone_tgt_df.reset_index(drop=True)
print("Target Set: ",abalone_tgt_df.shape)

abalone_source_df = AbaloneData_df.loc[(AbaloneData_df['Whole_weight'] > 0.5) & (AbaloneData_df['Whole_weight'] <= 1.0)]
abalone_source_df = abalone_source_df.drop(drop_col_abalone, axis = 1)
abalone_source_df = abalone_source_df.reset_index(drop=True)
print("Source Set: ",abalone_source_df.shape)

abalone_test_df = AbaloneData_df.loc[(AbaloneData_df['Whole_weight'] > 1.0)]
abalone_test_df = abalone_test_df.drop(drop_col_abalone, axis = 1)
abalone_test_df = abalone_test_df.reset_index(drop=True)
print("Test Set: ",abalone_test_df.shape)

#################### Splitting into features and target ####################
target_column_abalone = ['Rings']

abalone_tgt_df_y = abalone_tgt_df[target_column_abalone]
abalone_tgt_df_X = abalone_tgt_df.drop(target_column_abalone, axis = 1)

abalone_source_df_y = abalone_source_df[target_column_abalone]
abalone_source_df_X = abalone_source_df.drop(target_column_abalone, axis = 1)

abalone_test_df_y = abalone_test_df[target_column_abalone]
abalone_test_df_X = abalone_test_df.drop(target_column_abalone, axis = 1)

############## Merging the datasets ##########################################
abalone_X_df = pd.concat([abalone_tgt_df_X, abalone_source_df_X], ignore_index=True)
abalone_y_df = pd.concat([abalone_tgt_df_y, abalone_source_df_y], ignore_index=True)

abalone_np_train_X = abalone_X_df.to_numpy()
abalone_np_train_y = abalone_y_df.to_numpy()

abalone_np_test_X = abalone_test_df_X.to_numpy()
abalone_np_test_y = abalone_test_df_y.to_numpy()

abalone_np_train_y_list = abalone_np_train_y.ravel()
abalone_np_test_y_list = abalone_np_test_y.ravel()

src_size_abalone = len(abalone_source_df_y)
tgt_size_abalone = len(abalone_tgt_df_y)

print("---------------------------")

###############################################################################################################################################

Abalone Data
(4177, 9)
Target Set:  (1241, 8)
Source Set:  (1465, 8)
Test Set:  (1471, 8)
---------------------------


In [11]:
############# Transfer Learning specifications #########################
src_idx_abalone = np.arange(start=0, stop=(src_size_abalone - 1), step=1)
tgt_idx_abalone = np.arange(start=src_size_abalone, stop=((src_size_abalone + tgt_size_abalone)-1), step=1)

def get_estimator(**kwargs):
    return DecisionTreeRegressor(max_depth = 6)

kwargs_TwoTrAda = {'steps': 30,
                    'fold': 10,
                  'learning_rate': 0.1}

In [12]:
########################### TwoStagetrAdaBoostR2 Abalone #######################################

model_TwoTrAda_abalone = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 1000, cv=10) #, kwargs_TwoTrAda)
model_TwoTrAda_abalone.fit(abalone_np_train_X, abalone_np_train_y_list, src_idx_abalone, tgt_idx_abalone)

print("Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

y_pred_TwoTrAda_abalone = model_TwoTrAda_abalone.predict(abalone_np_test_X)
mse_TwoTrAda_abalone = sqrt(mean_squared_error(abalone_np_test_y, y_pred_TwoTrAda_abalone))
print("RMSE of Two-Stage TrAdaboost.R2:", mse_TwoTrAda_abalone)

r2_score_TwoTrAda_abalone = pearsonr(abalone_np_test_y_list, y_pred_TwoTrAda_abalone)
r2_score_TwoTrAda_abalone = (r2_score_TwoTrAda_abalone[0])**2
print("R^2 of TrAdaboostR2:", r2_score_TwoTrAda_abalone)

print("-------------------------------------------")

Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set val

In [73]:
########################### AdaBoostR2 Transfer Learning Abalone #####################################################
print("Adaboost.R2 Transfer Learning")
print("-------------------------------------------")

model_AdaTr_abalone = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 8), learning_rate=0.01, n_estimators=500) 
model_AdaTr_abalone.fit(abalone_np_train_X, abalone_np_train_y_list)

y_pred_AdaTr_abalone = model_AdaTr_abalone.predict(abalone_np_test_X)
mse_AdaTr_abalone = sqrt(mean_squared_error(abalone_np_test_y, y_pred_AdaTr_abalone))
print("RMSE of Adaboost.R2:", mse_AdaTr_abalone)

r2_score_AdaTr_abalone = pearsonr(abalone_np_test_y_list, y_pred_AdaTr_abalone)
r2_score_AdaTr_abalone = (r2_score_AdaTr_abalone[0])**2
print("R^2 of TrAdaboostR2:", r2_score_AdaTr_abalone)

print("-------------------------------------------")

Adaboost.R2 Transfer Learning
-------------------------------------------
RMSE of Adaboost.R2: 3.0283148106334967
R^2 of TrAdaboostR2: 0.149804197168237
-------------------------------------------


In [74]:
########################### AdaBoostR2 Abalone #####################################################
print("Adaboost.R2")
print("-------------------------------------------")


model_Ada_abalone = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 8), learning_rate=0.01, n_estimators=500) 
model_Ada_abalone.fit(abalone_np_train_X, abalone_np_train_y_list)

y_pred_ada_abalone = model_Ada_abalone.predict(abalone_np_test_X)
mse_Ada_abalone = sqrt(mean_squared_error(abalone_np_test_y, y_pred_ada_abalone))
print("RMSE of Adaboost.R2:", mse_Ada_abalone)

r2_score_Ada_abalone = pearsonr(abalone_np_test_y_list, y_pred_ada_abalone)
r2_score_Ada_abalone = (r2_score_Ada_abalone[0])**2
print("R^2 of TrAdaboostR2:", r2_score_Ada_abalone)

print("-------------------------------------------")

Adaboost.R2
-------------------------------------------
RMSE of Adaboost.R2: 3.0298705815237588
R^2 of TrAdaboostR2: 0.14310861479304007
-------------------------------------------


In [76]:
######################### Kernel Mean Matching Abalone #######################################
print("Kernel Mean Matching")
print("-------------------------------------------")

model_KMM_abalone = KMM(get_estimator = get_estimator)
model_KMM_abalone.fit(abalone_np_train_X, abalone_np_train_y_list, src_idx_abalone, tgt_idx_abalone)

y_pred_KMM_abalone = model_KMM_elevators.predict(abalone_df_test_X) ##Using dataframe instead of the numpy matrix
mse_KMM_abalone = sqrt(mean_squared_error(abalone_np_test_y, y_pred_KMM_abalone))
print("RMSE of KMM:", mse_KMM_abalone)

r2_score_KMM_abalone = pearsonr(abalone_np_test_y_list, y_pred_KMM_abalone)
r2_score_KMM_abalone = (r2_score_KMM_abalone[0])**2
print("R^2 of TrAdaboostR2:", r2_score_KMM_abalone)

#model_KMM_concrete.weights_   #To get the weights of the algorithm

print("-------------------------------------------")

Kernel Mean Matching
-------------------------------------------


KeyboardInterrupt: 

In [79]:
######################### Gradient Boosting Regression Transfer Learning Abalone #######################################

print("Gradient Boosting Regression Transfer Learning")
print("-------------------------------------------")


model_GBRTL_abalone = GradientBoostingRegressor(learning_rate=0.01, max_depth=4, n_estimators=1000, subsample=0.5)
model_GBRTL_abalone.fit(abalone_np_train_X, abalone_np_train_y_list)

y_pred_GBRTL_abalone = model_GBRTL_abalone.predict(abalone_test_df_X) ##Using dataframe instead of the numpy matrix
mse_GBRTL_abalone = sqrt(mean_squared_error(abalone_np_test_y, y_pred_GBRTL_abalone))
print("RMSE of KMM:", mse_GBRTL_abalone)

r2_score_GBRTL_abalone = pearsonr(abalone_np_test_y_list, y_pred_GBRTL_abalone)
r2_score_GBRTL_abalone = (r2_score_GBRTL_abalone[0])**2
print("R^2 of TrAdaboostR2:", r2_score_GBRTL_abalone)

print("-------------------------------------------")

Gradient Boosting Regression Transfer Learning
-------------------------------------------
RMSE of KMM: 3.1659095016420817
R^2 of TrAdaboostR2: 0.035792803563995765
-------------------------------------------


In [110]:
######################### Gradient Boosting Regression PoleTelecomm #######################################

print("Gradient Boosting Regression")
print("-------------------------------------------")


model_GBR_abalone = GradientBoostingRegressor(learning_rate=0.01, max_depth=4, n_estimators=1000, subsample=0.5)
model_GBR_abalone.fit(abalone_tgt_df_X, abalone_tgt_df_y)

y_pred_GBR_abalone = model_GBR_abalone.predict(abalone_test_df_X) ##Using dataframe instead of the numpy matrix
mse_GBR_abalone = sqrt(mean_squared_error(abalone_np_test_y, y_pred_GBR_abalone))
print("RMSE of KMM:", mse_GBR_abalone)

r2_score_GBR_abalone = pearsonr(abalone_np_test_y_list, y_pred_GBR_abalone)
r2_score_GBR_abalone = (r2_score_GBR_abalone[0])**2
print("R^2 of TrAdaboostR2:", r2_score_GBR_abalone)

print("-------------------------------------------")

Gradient Boosting Regression
-------------------------------------------


/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


RMSE of KMM: 3.043731919181302
R^2 of TrAdaboostR2: 0.010571438214392102
-------------------------------------------


In [ ]:
############################################ Transformed TrAdaBoost.R2 Abalone ########################################################

weights_KMM = model_KMM_abalone.weights_

val_mean = np.mean(weights_KMM)
weights_KMM = np.append(weights_KMM, val_mean)

abalone_source_df_X_trans = abalone_source_df_X.apply(lambda abalone_source_df_X: abalone_source_df_X * weights_KMM)
#concrete_source_df_X.multiply(weights_KMM, axis=0)

# concrete_np_train_X_trans = concrete_source_df_X_trans.to_numpy()

abalone_X_df = pd.concat([abalone_tgt_df_X, abalone_source_df_X_trans], ignore_index=True)
abalone_y_df = pd.concat([abalone_tgt_df_y, abalone_source_df_y], ignore_index=True)

abalone_np_train_X = abalone_X_df.to_numpy()
abalone_np_train_y = abalone_y_df.to_numpy()

abalone_np_test_X = abalone_test_df_X.to_numpy()
abalone_np_test_y = abalone_test_df_y.to_numpy()

abalone_np_train_y_list = abalone_np_train_y.ravel()
abalone_np_test_y_list = abalone_np_test_y.ravel()

# print(concrete_np_train_y_list.shape, concrete_np_train_X_trans.shape)

##################################################################
model_TwoTrAda_abalone = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 100, cv=10) #, kwargs_TwoTrAda)
model_TwoTrAda_abalone.fit(abalone_np_train_X, abalone_np_train_y_list, src_idx_abalone, tgt_idx_abalone)

print("Transformed Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

y_pred_TwoTrAda_abalone = model_TwoTrAda_abalone.predict(abalone_np_test_X)

mse_TwoTrAda_abalone = sqrt(mean_squared_error(abalone_np_test_y, y_pred_TwoTrAda_abalone))
print("RMSE of Two-Stage TrAdaboost.R2:", mse_TwoTrAda_abalone)

r2_score_TwoTrAda_abalone = pearsonr(abalone_np_test_y_list, y_pred_TwoTrAda_abalone)
r2_score_TwoTrAda_abalone = (r2_score_TwoTrAda_abalone[0])**2
print("R^2 of TrAdaboostR2:", r2_score_TwoTrAda_abalone)

print("-------------------------------------------")

In [81]:
#################################### STrAdaBoost.R2 Abalone ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

sample_size = [len(abalone_tgt_df_X), len(abalone_source_df_X)]
n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

model_stradaboost_abalone = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6),
                    n_estimators = n_estimators, sample_size = sample_size,
                    steps = steps, fold = fold, random_state = random_state)


model_stradaboost_abalone.fit(abalone_np_train_X, abalone_np_train_y_list)
y_pred_stradaboost_abalone = model_stradaboost_abalone.predict(abalone_np_test_X)

mse_stradaboost_abalone = sqrt(mean_squared_error(abalone_np_test_y, y_pred_stradaboost_abalone))
print("RMSE of STrAdaboostR2:", mse_stradaboost_abalone)

r2_score_stradaboost_abalone = pearsonr(abalone_np_test_y_list, y_pred_stradaboost_abalone)
r2_score_stradaboost_abalone = (r2_score_stradaboost_abalone[0])**2
print("R^2 of STrAdaboostR2:", r2_score_stradaboost_abalone)

Inside STrAdaBoost.R2
RMSE of STrAdaboostR2: 3.091666842196879
R^2 of STrAdaboostR2: 0.030949749633660227


In [113]:
################################################## Kinematics ######################################################################
#### range: [0.04 - 1.45]
#### Mid of correlation variable: theta7
#### [0, 0.6] [0.6, 0.85], [0.6, 0.85]
####################################################################################################################################
target_var_Kinematics = ['y']
colnames_Kinematics = ['theta1', 'theta2', 'theta3', 'theta4', 'theta5', 'theta6', 'theta7', 'theta8', 'y']
KinematicsData_df = pd.read_csv('UCI_regression/Kinematics/kin8nm.data', header = None,  names = colnames_Kinematics)
print("Kinematics Data")
print(KinematicsData_df.shape)

########## Corr Kinematics ################
# print("The correlation matrix is: ")
# print(KinematicsData_df.corr().sort_values('y')['y'])

##################### Splitting in 3 equal parts #######################################
# print(KinematicsData_df.sort_values('theta7')['theta7'])

drop_col_kinematics = ['theta7']

kinematics_tgt_df = KinematicsData_df.loc[(KinematicsData_df['theta7'] <= -0.5)]
kinematics_tgt_df = kinematics_tgt_df.drop(drop_col_kinematics, axis = 1)
kinematics_tgt_df = kinematics_tgt_df.reset_index(drop=True)
print("Target Set: ",kinematics_tgt_df.shape)


kinematics_source_df = KinematicsData_df.loc[(KinematicsData_df['theta7'] > -0.5) & (KinematicsData_df['theta7'] <= 0.5)]
kinematics_source_df = kinematics_source_df.drop(drop_col_kinematics, axis = 1)
kinematics_source_df = kinematics_source_df.reset_index(drop=True)
print("Source Set: ",kinematics_source_df.shape)


kinematics_test_df = KinematicsData_df.loc[(KinematicsData_df['theta7'] > 0.5)]
kinematics_test_df = kinematics_test_df.drop(drop_col_kinematics, axis = 1)
kinematics_test_df = kinematics_test_df.reset_index(drop=True)
print("Test Set: ",kinematics_test_df.shape)


# #################### Splitting with small target set and large source and test set #############
# KinematicsData_df_remain, KinematicsData_df_tgt = train_test_split(KinematicsData_df, test_size = 0.05) ## test_size = tgt size
# KinematicsData_df_source, KinematicsData_df_test = train_test_split(KinematicsData_df_remain, test_size = 0.3) ## test_size = tgt size
# print(KinematicsData_df_tgt.shape, KinematicsData_df_source.shape, KinematicsData_df_test.shape)

#################### Splitting into features and target ####################
target_column_kinematics = ['y']

kinematics_tgt_df_y = kinematics_tgt_df[target_column_kinematics]
kinematics_tgt_df_X = kinematics_tgt_df.drop(target_column_kinematics, axis = 1)

kinematics_source_df_y = kinematics_source_df[target_column_kinematics]
kinematics_source_df_X = kinematics_source_df.drop(target_column_kinematics, axis = 1)

kinematics_test_df_y = kinematics_test_df[target_column_kinematics]
kinematics_test_df_X = kinematics_test_df.drop(target_column_kinematics, axis = 1)

############## Merging the datasets ##########################################
kinematics_X_df = pd.concat([kinematics_tgt_df_X, kinematics_source_df_X], ignore_index=True)
kinematics_y_df = pd.concat([kinematics_tgt_df_y, kinematics_source_df_y], ignore_index=True)

kinematics_np_train_X = kinematics_X_df.to_numpy()
kinematics_np_train_y = kinematics_y_df.to_numpy()

kinematics_np_test_X = kinematics_test_df_X.to_numpy()
kinematics_np_test_y = kinematics_test_df_y.to_numpy()

kinematics_np_train_y_list = kinematics_np_train_y.ravel()
kinematics_np_test_y_list = kinematics_np_test_y.ravel()

src_size_kinematics = len(kinematics_source_df_y)
tgt_size_kinematics = len(kinematics_tgt_df_y)

print("---------------------------")

###############################################################################################################################################

Kinematics Data
(8192, 9)
Target Set:  (2755, 8)
Source Set:  (2598, 8)
Test Set:  (2839, 8)
---------------------------


In [114]:
############# Transfer Learning specifications #########################
src_idx_kinematics = np.arange(start=0, stop=(src_size_kinematics - 1), step=1)
src_idx_kinematics = np.arange(start=src_size_kinematics, stop=((src_size_kinematics + tgt_size_kinematics)-1), step=1)
src_idx_kinematics, src_idx_kinematics 

(array([2598, 2599, 2600, ..., 5349, 5350, 5351]),
 array([2598, 2599, 2600, ..., 5349, 5350, 5351]))

In [115]:
########################### TwoStagetrAdaBoostR2 Kinematics #######################################

model_TwoTrAda_kinematics = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 1000, cv=10) #, kwargs_TwoTrAda)
model_TwoTrAda_kinematics.fit(kinematics_np_train_X, kinematics_np_train_y_list, src_idx_kinematics, tgt_idx_kinematics)

print("Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

y_pred_TwoTrAda_kinematics = model_TwoTrAda_kinematics.predict(kinematics_np_test_X)
mse_TwoTrAda_kinematics = sqrt(mean_squared_error(kinematics_np_test_y, y_pred_TwoTrAda_kinematics))
print("RMSE of Two-Stage TrAdaboost.R2:", mse_TwoTrAda_kinematics)

r2_score_TwoTrAda_kinematics = pearsonr(kinematics_np_test_y_list, y_pred_TwoTrAda_kinematics)
r2_score_TwoTrAda_kinematics = (r2_score_TwoTrAda_kinematics[0])**2
print("R^2 of TrAdaboostR2:", r2_score_TwoTrAda_kinematics)

print("-------------------------------------------")

TypeError: __init__() got an unexpected keyword argument 'get_estimator'

In [84]:
########################### AdaBoostR2 Transfer Learning Kinematics #####################################################
print("Adaboost.R2 Transfer Learning")
print("-------------------------------------------")

model_Ada_kinematics = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 8), learning_rate=0.01, n_estimators=500) 
model_Ada_kinematics.fit(kinematics_np_train_X, kinematics_np_train_y_list)

y_pred_ada_kinematics = model_Ada_kinematics.predict(kinematics_np_test_X)
mse_Ada_kinematics = sqrt(mean_squared_error(kinematics_np_test_y, y_pred_ada_kinematics))
print("RMSE of Adaboost.R2:", mse_Ada_kinematics)

r2_score_Ada_kinematics = pearsonr(kinematics_np_test_y_list, y_pred_ada_kinematics)
r2_score_Ada_kinematics = (r2_score_Ada_kinematics[0])**2
print("R^2 of TrAdaboostR2:", r2_score_Ada_kinematics)

print("-------------------------------------------")

Adaboost.R2 Transfer Learning
-------------------------------------------
RMSE of Adaboost.R2: 0.2604143796356598
R^2 of TrAdaboostR2: 0.2228593521941291
-------------------------------------------


In [85]:
########################### AdaBoostR2 Kinematics #####################################################
print("Adaboost.R2")
print("-------------------------------------------")


model_Ada_kinematics = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 8), learning_rate=0.01, n_estimators=500) 
model_Ada_kinematics.fit(kinematics_np_train_X, kinematics_np_train_y_list)

y_pred_ada_kinematics = model_Ada_kinematics.predict(kinematics_np_test_X)
mse_Ada_kinematics = sqrt(mean_squared_error(kinematics_np_test_y, y_pred_ada_kinematics))
print("RMSE of Adaboost.R2:", mse_Ada_kinematics)

r2_score_Ada_kinematics = pearsonr(kinematics_np_test_y_list, y_pred_ada_kinematics)
r2_score_Ada_kinematics = (r2_score_Ada_kinematics[0])**2
print("R^2 of TrAdaboostR2:", r2_score_Ada_kinematics)

print("-------------------------------------------")

Adaboost.R2
-------------------------------------------
RMSE of Adaboost.R2: 0.2607164590801618
R^2 of TrAdaboostR2: 0.22154664510146269
-------------------------------------------


In [ ]:
######################### Kernel Mean Matching Kinematics #######################################
print("Kernel Mean Matching")
print("-------------------------------------------")

model_KMM_kinematics = KMM(get_estimator = get_estimator)
model_KMM_kinematics.fit(kinematics_np_train_X, kinematics_np_train_y_list, src_idx_kinematics, tgt_idx_kinematics)

y_pred_KMM_kinematics = model_KMM_kinematics.predict(kinematics_df_test_X) ##Using dataframe instead of the numpy matrix
mse_KMM_kinematics = sqrt(mean_squared_error(kinematics_np_test_y, y_pred_KMM_kinematics))
print("RMSE of KMM:", mse_KMM_kinematics)

r2_score_KMM_kinematics = pearsonr(kinematics_np_test_y_list, y_pred_KMM_kinematics)
r2_score_KMM_kinematics = (r2_score_KMM_kinematics[0])**2
print("R^2 of TrAdaboostR2:", r2_score_KMM_kinematics)

#model_KMM_concrete.weights_   #To get the weights of the algorithm

print("-------------------------------------------")

In [105]:
######################### Gradient Boosting Regression Transfer Learning Kinematics #######################################

print("Gradient Boosting Regression Transfer Learning")
print("-------------------------------------------")


model_GBRTL_kinematics = GradientBoostingRegressor(learning_rate=0.01, max_depth=4, n_estimators=1000, subsample=0.5)
model_GBRTL_kinematics.fit(kinematics_np_train_X, kinematics_np_train_y_list)

y_pred_GBRTL_kinematics = model_GBRTL_kinematics.predict(kinematics_test_df_X) ##Using dataframe instead of the numpy matrix
mse_GBRTL_kinematics = sqrt(mean_squared_error(kinematics_np_test_y, y_pred_GBRTL_kinematics))
print("RMSE of KMM:", mse_GBRTL_kinematics)

r2_score_GBRTL_kinematics = pearsonr(kinematics_np_test_y_list, y_pred_GBRTL_kinematics)
r2_score_GBRTL_kinematics = (r2_score_GBRTL_kinematics[0])**2
print("R^2 of TrAdaboostR2:", r2_score_GBRTL_kinematics)

print("-------------------------------------------")

Gradient Boosting Regression Transfer Learning
-------------------------------------------
RMSE of KMM: 0.25586133995157134
R^2 of TrAdaboostR2: 0.2538128608441631
-------------------------------------------


In [111]:
######################### Gradient Boosting Regression Kinematics #######################################

print("Gradient Boosting Regression")
print("-------------------------------------------")


model_GBR_kinematics = GradientBoostingRegressor(learning_rate=0.01, max_depth=4, n_estimators=1000, subsample=0.5)
model_GBR_kinematics.fit(kinematics_tgt_df_X, kinematics_tgt_df_y)

y_pred_GBR_kinematics = model_GBR_kinematics.predict(kinematics_test_df_X) ##Using dataframe instead of the numpy matrix
mse_GBR_kinematics = sqrt(mean_squared_error(kinematics_np_test_y, y_pred_GBR_kinematics))
print("RMSE of KMM:", mse_GBR_kinematics)

r2_score_GBR_kinematics = pearsonr(kinematics_np_test_y_list, y_pred_GBR_kinematics)
r2_score_GBR_kinematics = (r2_score_GBR_kinematics[0])**2
print("R^2 of TrAdaboostR2:", r2_score_GBR_kinematics)

print("-------------------------------------------")

Gradient Boosting Regression
-------------------------------------------


/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


RMSE of KMM: 0.30639546537425716
R^2 of TrAdaboostR2: 0.06941572878514127
-------------------------------------------


In [ ]:
################ Transformed TrAdaBoost.R2 Kinematics #############################################

weights_KMM = model_KMM_kinematics.weights_

val_mean = np.mean(weights_KMM)
weights_KMM = np.append(weights_KMM, val_mean)

kinematics_source_df_X_trans = kinematics_source_df_X.apply(lambda kinematics_source_df_X: kinematics_source_df_X * weights_KMM)
#concrete_source_df_X.multiply(weights_KMM, axis=0)

# concrete_np_train_X_trans = concrete_source_df_X_trans.to_numpy()

kinematics_X_df = pd.concat([kinematics_tgt_df_X, kinematics_source_df_X_trans], ignore_index=True)
kinematics_y_df = pd.concat([kinematics_tgt_df_y, kinematics_source_df_y], ignore_index=True)

kinematics_np_train_X = kinematics_X_df.to_numpy()
kinematics_np_train_y = kinematics_y_df.to_numpy()

kinematics_np_test_X = kinematics_test_df_X.to_numpy()
kinematics_np_test_y = kinematics_test_df_y.to_numpy()

kinematics_np_train_y_list = kinematics_np_train_y.ravel()
kinematics_np_test_y_list = kinematics_np_test_y.ravel()

# print(concrete_np_train_y_list.shape, concrete_np_train_X_trans.shape)

##################################################################
model_TwoTrAda_kinematics = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 100, cv=10) #, kwargs_TwoTrAda)
model_TwoTrAda_kinematics.fit(kinematics_np_train_X, kinematics_np_train_y_list, src_idx_kinematics, tgt_idx_kinematics)

print("Transformed Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

y_pred_TwoTrAda_kinematics = model_TwoTrAda_kinematics.predict(kinematics_np_test_X)

mse_TwoTrAda_kinematics = sqrt(mean_squared_error(kinematics_np_test_y, y_pred_TwoTrAda_kinematics))
print("RMSE of Two-Stage TrAdaboost.R2:", mse_TwoTrAda_kinematics)

r2_score_TwoTrAda_kinematics = pearsonr(kinematics_np_test_y_list, y_pred_TwoTrAda_kinematics)
r2_score_TwoTrAda_kinematics = (r2_score_TwoTrAda_kinematics[0])**2
print("R^2 of TrAdaboostR2:", r2_score_TwoTrAda_kinematics)

print("-------------------------------------------")

In [88]:
#################################### STrAdaBoost.R2 Kinematics ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

sample_size = [len(kinematics_tgt_df_X), len(kinematics_source_df_X)]
n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

model_stradaboost_kinematics = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6),
                    n_estimators = n_estimators, sample_size = sample_size,
                    steps = steps, fold = fold, random_state = random_state)


model_stradaboost_kinematics.fit(kinematics_np_train_X, kinematics_np_train_y_list)
y_pred_stradaboost_kinematics = model_stradaboost_kinematics.predict(kinematics_np_test_X)

mse_stradaboost_kinematics = sqrt(mean_squared_error(kinematics_np_test_y, y_pred_stradaboost_kinematics))
print("RMSE of STrAdaboostR2:", mse_stradaboost_kinematics)

r2_score_stradaboost_kinematics = pearsonr(kinematics_np_test_y_list, y_pred_stradaboost_kinematics)
r2_score_stradaboost_kinematics = (r2_score_stradaboost_kinematics[0])**2
print("R^2 of STrAdaboostR2:", r2_score_stradaboost_kinematics)

Inside STrAdaBoost.R2
RMSE of STrAdaboostR2: 0.23854389626490993
R^2 of STrAdaboostR2: 0.2799693293860748


In [4]:
################################### Computer Activity ###########################################
colnames_CompAct = ['lread', 'lwrite', 'scall', 'sread', 'swrite', 'fork', 'exec', 'rchar', 'wchar', 'pgout', 'ppgout', 
                    'pgfree', 'pgscan', 'atch', 'pgin', 'ppgin', 'pflt', 'vflt', 'runqsz', 'freemem', 'freeswap', 'usr' ]
CompActData_df = pd.read_csv('UCI_regression/ComputerActivity/cpu_act.data', header = None, names = colnames_CompAct)
print("Computer Activity Data")
print(CompActData_df.shape)

########## Corr Computer Activity ################
# print("The correlation matrix is: ")
# print(CompActData_df.corr().sort_values('usr')['usr'])

##################### Splitting in 3 equal parts #######################################
# print(CompActData_df.sort_values('pgin')['pgin'])

drop_col_compact = ['pgin']

compact_tgt_df = CompActData_df.loc[(CompActData_df['pgin'] <= 1.0)]
compact_tgt_df = compact_tgt_df.drop(drop_col_compact, axis = 1)
compact_tgt_df = compact_tgt_df.reset_index(drop=True)
print("Target Set: ",compact_tgt_df.shape)


compact_source_df = CompActData_df.loc[(CompActData_df['pgin'] > 1.0) & (CompActData_df['pgin'] <= 6.0)]
compact_source_df = compact_source_df.drop(drop_col_compact, axis = 1)
compact_source_df = compact_source_df.reset_index(drop=True)
print("Source Set: ",compact_source_df.shape)


compact_test_df = CompActData_df.loc[(CompActData_df['pgin'] > 6.0)]
compact_test_df = compact_test_df.drop(drop_col_compact, axis = 1)
compact_test_df = compact_test_df.reset_index(drop=True)
print("Test Set: ",compact_test_df.shape)


#################### Splitting with small target set and large source and test set #############
# KinematicsData_df_remain, KinematicsData_df_tgt = train_test_split(KinematicsData_df, test_size = 0.05) ## test_size = tgt size
# KinematicsData_df_source, KinematicsData_df_test = train_test_split(KinematicsData_df_remain, test_size = 0.3) ## test_size = tgt size
# print(KinematicsData_df_tgt.shape, KinematicsData_df_source.shape, KinematicsData_df_test.shape)

#################### Splitting into features and target ####################
target_column_compact = ['usr']

compact_tgt_df_y = compact_tgt_df[target_column_compact]
compact_tgt_df_X = compact_tgt_df.drop(target_column_compact, axis = 1)

compact_source_df_y = compact_source_df[target_column_compact]
compact_source_df_X = compact_source_df.drop(target_column_compact, axis = 1)

compact_test_df_y = compact_test_df[target_column_compact]
compact_test_df_X = compact_test_df.drop(target_column_compact, axis = 1)

############## Merging the datasets ##########################################
compact_X_df = pd.concat([compact_tgt_df_X, compact_source_df_X], ignore_index=True)
compact_y_df = pd.concat([compact_tgt_df_y, compact_source_df_y], ignore_index=True)

compact_np_train_X = compact_X_df.to_numpy()
compact_np_train_y = compact_y_df.to_numpy()

compact_np_test_X = compact_test_df_X.to_numpy()
compact_np_test_y = compact_test_df_y.to_numpy()

compact_np_train_y_list = compact_np_train_y.ravel()
compact_np_test_y_list = compact_np_test_y.ravel()

src_size_compact = len(compact_source_df_y)
tgt_size_compact = len(compact_tgt_df_y)


print("---------------------------")

###############################################################################################################################################

Computer Activity Data
(8192, 22)
Target Set:  (2766, 21)
Source Set:  (2539, 21)
Test Set:  (2887, 21)
---------------------------


In [7]:
############# Transfer Learning specifications #########################
src_idx_compact = np.arange(start=0, stop=(src_size_compact - 1), step=1)
src_idx_compact = np.arange(start=src_size_compact, stop=((src_size_compact + tgt_size_compact)-1), step=1)
src_idx_compact, src_idx_compact 

(array([2539, 2540, 2541, ..., 5301, 5302, 5303]),
 array([2539, 2540, 2541, ..., 5301, 5302, 5303]))

In [ ]:
########################### TwoStagetrAdaBoostR2 compAct #######################################

model_TwoTrAda_compact = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 1000, cv=10) #, kwargs_TwoTrAda)
model_TwoTrAda_compact.fit(compact_np_train_X, compact_np_train_y_list, src_idx_compact, tgt_idx_compact)

print("Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

y_pred_TwoTrAda_compact = model_TwoTrAda_compact.predict(compact_np_test_X)
mse_TwoTrAda_compact = sqrt(mean_squared_error(compact_np_test_y, y_pred_TwoTrAda_compact))
print("RMSE of Two-Stage TrAdaboost.R2:", mse_TwoTrAda_compact)

r2_score_TwoTrAda_compact = pearsonr(compact_np_test_y_list, y_pred_TwoTrAda_compact)
r2_score_TwoTrAda_compact = (r2_score_TwoTrAda_compact[0])**2
print("R^2 of TrAdaboostR2:", r2_score_TwoTrAda_compact)

print("-------------------------------------------")

In [91]:
########################### AdaBoostR2 Transfer Learning compAct #####################################################
print("Adaboost.R2 Transfer Learning")
print("-------------------------------------------")

model_Ada_compact = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 8), learning_rate=0.01, n_estimators=500) 
model_Ada_compact.fit(compact_np_train_X, compact_np_train_y_list)

y_pred_ada_compact = model_Ada_compact.predict(compact_np_test_X)
mse_Ada_compact = sqrt(mean_squared_error(compact_np_test_y, y_pred_ada_compact))
print("RMSE of Adaboost.R2:", mse_Ada_compact)

r2_score_Ada_compact = pearsonr(compact_np_test_y_list, y_pred_ada_compact)
r2_score_Ada_compact = (r2_score_Ada_compact[0])**2
print("R^2 of TrAdaboostR2:", r2_score_Ada_compact)

print("-------------------------------------------")

Adaboost.R2 Transfer Learning
-------------------------------------------
RMSE of Adaboost.R2: 3.9460278220363523
R^2 of TrAdaboostR2: 0.970225964792705
-------------------------------------------


In [93]:
########################### AdaBoostR2 compAct #####################################################
print("Adaboost.R2")
print("-------------------------------------------")


model_Ada_compact = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 8), learning_rate=0.01, n_estimators=500) 
model_Ada_compact.fit(compact_np_train_X, compact_np_train_y_list)

y_pred_ada_compact = model_Ada_compact.predict(compact_np_test_X)
mse_Ada_compact = sqrt(mean_squared_error(compact_np_test_y, y_pred_ada_compact))
print("RMSE of Adaboost.R2:", mse_Ada_compact)

r2_score_Ada_compact = pearsonr(compact_np_test_y_list, y_pred_ada_compact)
r2_score_Ada_compact = (r2_score_Ada_compact[0])**2
print("R^2 of TrAdaboostR2:", r2_score_Ada_compact)

print("-------------------------------------------")

Adaboost.R2
-------------------------------------------
RMSE of Adaboost.R2: 3.933592655266186
R^2 of TrAdaboostR2: 0.9703048050814059
-------------------------------------------


In [ ]:
######################### Kernel Mean Matching compAct #######################################
print("Kernel Mean Matching")
print("-------------------------------------------")

model_KMM_compact = KMM(get_estimator = get_estimator)
model_KMM_compact.fit(kinematics_np_train_X, kinematics_np_train_y_list, src_idx_kinematics, tgt_idx_kinematics)

y_pred_KMM_compact = model_KMM_compact.predict(kinematics_df_test_X) ##Using dataframe instead of the numpy matrix
mse_KMM_compact = sqrt(mean_squared_error(compact_np_test_y, y_pred_KMM_compact))
print("RMSE of KMM:", mse_KMM_compact)

r2_score_KMM_compact = pearsonr(compact_np_test_y_list, y_pred_KMM_compact)
r2_score_KMM_compact = (r2_score_KMM_compact[0])**2
print("R^2 of TrAdaboostR2:", r2_score_KMM_compact)

#model_KMM_concrete.weights_   #To get the weights of the algorithm

print("-------------------------------------------")

In [94]:
######################### Gradient Boosting Regression Transfer Learning compAct #######################################

print("Gradient Boosting Regression Transfer Learning")
print("-------------------------------------------")


model_GBRTL_compact = GradientBoostingRegressor(learning_rate=0.01, max_depth=4, n_estimators=1000, subsample=0.5)
model_GBRTL_compact.fit(compact_np_train_X, compact_np_train_y_list)

y_pred_GBRTL_compact = model_GBRTL_compact.predict(compact_test_df_X) ##Using dataframe instead of the numpy matrix
mse_GBRTL_compact = sqrt(mean_squared_error(compact_np_test_y, y_pred_GBRTL_compact))
print("RMSE of KMM:", mse_GBRTL_compact)

r2_score_GBRTL_compact = pearsonr(compact_np_test_y_list, y_pred_GBRTL_compact)
r2_score_GBRTL_compact = (r2_score_GBRTL_compact[0])**2
print("R^2 of TrAdaboostR2:", r2_score_GBRTL_compact)

print("-------------------------------------------")

Gradient Boosting Regression Transfer Learning
-------------------------------------------
RMSE of KMM: 3.2122237003651817
R^2 of TrAdaboostR2: 0.9761264178149078
-------------------------------------------


In [112]:
######################### Gradient Boosting Regression compAct #######################################

print("Gradient Boosting Regression")
print("-------------------------------------------")


model_GBR_compact = GradientBoostingRegressor(learning_rate=0.01, max_depth=4, n_estimators=1000, subsample=0.5)
model_GBR_compact.fit(compact_tgt_df_X, compact_tgt_df_y)

y_pred_GBR_compact = model_GBR_compact.predict(compact_test_df_X) ##Using dataframe instead of the numpy matrix
mse_GBR_compact = sqrt(mean_squared_error(compact_np_test_y, y_pred_GBR_compact))
print("RMSE of KMM:", mse_GBR_compact)

r2_score_GBR_compact = pearsonr(compact_np_test_y_list, y_pred_GBR_compact)
r2_score_GBR_compact = (r2_score_GBR_compact[0])**2
print("R^2 of TrAdaboostR2:", r2_score_GBR_compact)

print("-------------------------------------------")

Gradient Boosting Regression
-------------------------------------------


/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


RMSE of KMM: 4.103227316601275
R^2 of TrAdaboostR2: 0.9639686110913217
-------------------------------------------


In [ ]:
########################################## Transformed TrAdaBoost.R2 compAct ###############################################################

weights_KMM = model_KMM_compact.weights_

val_mean = np.mean(weights_KMM)
weights_KMM = np.append(weights_KMM, val_mean)

compact_source_df_X_trans = compact_source_df_X.apply(lambda compact_source_df_X: compact_source_df_X * weights_KMM)
#concrete_source_df_X.multiply(weights_KMM, axis=0)
# concrete_np_train_X_trans = concrete_source_df_X_trans.to_numpy()

compact_X_df = pd.concat([compact_tgt_df_X, compact_source_df_X_trans], ignore_index=True)
compact_y_df = pd.concat([compact_tgt_df_y, compact_source_df_y], ignore_index=True)

compact_np_train_X = compact_X_df.to_numpy()
compact_np_train_y = compact_y_df.to_numpy()

compact_np_test_X = compact_test_df_X.to_numpy()
compact_np_test_y = compact_test_df_y.to_numpy()

compact_np_train_y_list = compact_np_train_y.ravel()
compact_np_test_y_list = compact_np_test_y.ravel()

# print(concrete_np_train_y_list.shape, concrete_np_train_X_trans.shape)

##################################################################
model_TwoTrAda_compact = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 100, cv=10) #, kwargs_TwoTrAda)
model_TwoTrAda_compact.fit(compact_np_train_X, compact_np_train_y_list, src_idx_compact, tgt_idx_compact)

print("Transformed Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

y_pred_TwoTrAda_compact = model_TwoTrAda_kinematics.predict(compact_np_test_X)

mse_TwoTrAda_compact = sqrt(mean_squared_error(compact_np_test_y, y_pred_TwoTrAda_compact))
print("RMSE of Two-Stage TrAdaboost.R2:", mse_TwoTrAda_compact)

r2_score_TwoTrAda_compact = pearsonr(compact_np_test_y_list, y_pred_TwoTrAda_compact)
r2_score_TwoTrAda_compact = (r2_score_TwoTrAda_compact[0])**2
print("R^2 of TrAdaboostR2:", r2_score_TwoTrAda_compact)

print("-------------------------------------------")

In [95]:
#################################### STrAdaBoost.R2 compAct ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

sample_size = [len(compact_tgt_df_X), len(compact_source_df_X)]
n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

model_stradaboost_compact = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6),
                    n_estimators = n_estimators, sample_size = sample_size,
                    steps = steps, fold = fold, random_state = random_state)


model_stradaboost_compact.fit(compact_np_train_X, compact_np_train_y_list)
y_pred_stradaboost_compact = model_stradaboost_compact.predict(compact_np_test_X)

mse_stradaboost_compact = sqrt(mean_squared_error(compact_np_test_y, y_pred_stradaboost_compact))
print("RMSE of STrAdaboostR2:", mse_stradaboost_compact)

r2_score_stradaboost_compact = pearsonr(compact_np_test_y_list, y_pred_stradaboost_compact)
r2_score_stradaboost_compact = (r2_score_stradaboost_compact[0])**2
print("R^2 of STrAdaboostR2:", r2_score_stradaboost_compact)

Inside STrAdaBoost.R2
RMSE of STrAdaboostR2: 3.767416563517102
R^2 of STrAdaboostR2: 0.9679352328179658


In [ ]:
################################################## UNWANTED DATASETS ######################################################################

################################### FriedmanExample ###########################################
target_var_FE = ['y']
colnames_FE = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'y']
FriedmanData_df = pd.read_csv('UCI_regression/FriedmanExample/fried_delve.data', header = None) #, names = colnames_FE)
print("Friedman Data")
print(FriedmanData_df.shape)

########## Corr Friedman Example ################
# print("The correlation matrix is: ")
# print(FriedmanData_df)
# print(FriedmanData_df.corr()['y'])

print("---------------------------")
####################################################################################################################################

################################### Diabetes ###############################################
### Very small dataset, can be removed
#### Mid of correlation variable: , range: 
#### 
#######################################################################################
target_var_diabetes = ['c_peptide']
colnames_diabetes = ['age', 'deficit', 'c_peptide']
DiabetesData_df = pd.read_csv('UCI_regression/Diabetes/diabetes.data', header = None, names = colnames_diabetes)
print("Diabetes Data")
print(DiabetesData_df.shape)

######### Corr Diabetes ################
# print("The correlation matrix is: ")
# print(DiabetesData_df.corr()['c_peptide'])

#################### Splitting with small target set and large source and test set #############
# DiabetesData_df_remain, DiabetesData_df_tgt = train_test_split(DiabetesData_df, test_size = 0.025) ## test_size = tgt size
# DiabetesData_df_source, DiabetesData_df_test = train_test_split(DiabetesData_df_remain, test_size = 0.3) ## test_size = tgt size
# print(DiabetesData_df_tgt.shape, DiabetesData_df_source.shape, DiabetesData_df_test.shape)

print("---------------------------")
##############################################################################################################################################

# ################################### Pyrimidines ###########################################
# ###########################################################################################
# ### Very small dataset, can be removed
# #### Mid of correlation variable: , range: 
# #### 
# ###########################################################################################
# colnames_Pyrimidines = ['p1_polar', 'p1_size', 'p1_flex', 'p1_h_doner', 'p1_h_acceptor', 'p1_pi_doner', 'p1_pi_acceptor', 'p1_polarisable', 'p1_sigma',
#                         'p2_polar', 'p2_size', 'p2_flex', 'p2_h_doner', 'p2_h_acceptor', 'p2_pi_doner', 'p2_pi_acceptor', 'p2_polarisable', 'p2_sigma', 
#                         'p3_polar', 'p3_size', 'p3_flex', 'p3_h_doner', 'p3_h_acceptor', 'p3_pi_doner', 'p3_pi_acceptor', 'p3_polarisable', 'p3_sigma', 'activity']
# PyrimidinesData_df = pd.read_csv('UCI_regression/Pyrimidines/pyrim.data', header = None, names = colnames_Pyrimidines)
# print("Pyrimidines Data")
# print(PyrimidinesData_df.shape)
# print("---------------------------")

###############################################################################################################################################

# ################################### Machine-Cpu ##################################################
# #### range: [0 - 52]
# #### Mid of correlation variable: CHMIN
# #### 
# #### No correlation required since all attributes are integer variables
# ##############################################################################################################################################

################################### Stock ###########################################
colnames_Stock = ['company1', 'company2', 'company3', 'company4', 'company5', 'company6', 'company7', 'company8', 'company9', 'company10']
StockData_df = pd.read_csv('UCI_regression/Stock/stock.data', header = None, names = colnames_Stock) 
print("Stock Data")
print(StockData_df.shape)

############################### Corr Stock ##########################################
print("The correlation matrix is: ")
print(StockData_df.corr().sort_values('y')['y'])


print("---------------------------")

###############################################################################################################################################

In [42]:
###################################################### Regression Datasets ################################################################################

# colnames_MachineCPU = ['MYCT', 'MMIN', 'MMAX', 'CACH', 'CHMIN', 'CHMX', 'class']
# MachineCPUData_df = pd.read_csv('UCI_regression/Machine-Cpu/machine.data', header = None, names = colnames_MachineCPU)
# print("Machine CPU Data")
# print(MachineCPUData_df.shape)

# ########## Corr Machine-Cpu ################
# # print("The correlation matrix is: ")
# # print(MachineCPUData_df.corr().sort_values('class')['class'])

# ##################### Splitting in 3 equal parts #######################################
# # print(MachineCPUData_df.sort_values('CHMIN')['CHMIN'])
# # print(len(MachineCPUData_df.loc[(MachineCPUData_df['CHMIN'] <= 0)]))
# # print(len(MachineCPUData_df.loc[(MachineCPUData_df['y'] > 0.6) & (MachineCPUData_df['y'] <= 0.85)]))
# # print(len(MachineCPUData_df.loc[(MachineCPUData_df['y'] > 0.85)]))

# print("---------------------------")

###############################################################################################################################################

# ################################### Wisconsin Breast Cancer ###########################################
# colnames_WBC = ['Lymph_node', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 
#                   'concave_points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 
#                   'compactness_se', 'concavity_se', 'concave_points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst', 
#                   'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'concave_points_worst', 'symmetry_worst', 
#                   'fractal_dimension_worst', 'Tumor_size', 'Time']
# WBCData_df = pd.read_csv('UCI_regression/WiscoinBreastCancer/r_wpbc.data', header = None, names = colnames_WBC)
# print("Wisconsin Breast Cancer Data")
# print(WBCData_df.shape)
# print("---------------------------")

###############################################################################################################################################

############################# Preprocessing Data ################################################################
############ Finding the correlation first ####################
# print("The correlation matrix is: ")
# print(ConcreteData_df.corr())

# col = ConcreteData_df['Cement']
# ConcreteData_df = ConcreteData_df.sort_values(by=['Cement'])
# col = HousingData_df['nox']


# drop_col = ['Cement']
drop_col = ['horsepower']
# drop_col = ['nox']

# Train_df = ConcreteData_df.loc[(ConcreteData_df['Cement'] <= 225)]
# Train_df = HousingData_df.loc[(HousingData_df['nox'] > 0.475) & (HousingData_df['nox'] <= 0.600)]
Train_df = AutoData_df.loc[(AutoData_df['horsepower'] > 80) & (AutoData_df['horsepower'] <= 110)]

Train_df = Train_df.drop(drop_col, axis = 1)
Train_df = Train_df.reset_index(drop=True)
print(Train_df.shape)

# Source_df = ConcreteData_df.loc[(ConcreteData_df['Cement'] > 225) & (ConcreteData_df['Cement'] <= 350)]
# Source_df = HousingData_df.loc[(HousingData_df['nox'] <= 0.475)]
Source_df = AutoData_df.loc[(AutoData_df['horsepower'] <= 80)]

Source_df = Source_df.drop(drop_col, axis = 1)
Source_df = Source_df.reset_index(drop=True)
print(Source_df.shape)

# Test_df = ConcreteData_df.loc[(ConcreteData_df['Cement'] > 350)]
# Test_df = HousingData_df.loc[(HousingData_df['nox'] > 0.600)]
Test_df = AutoData_df.loc[(AutoData_df['horsepower'] > 110)]

Test_df = Test_df.drop(drop_col, axis = 1)
Test_df = Test_df.reset_index(drop=True)
print(Test_df.shape)

Housing Data
(506, 14)
---------------------------
Auto Data
(392, 8)
---------------------------
Abalone Data
(4177, 9)
(209, 9) (2777, 9) (1191, 9)
---------------------------
Friedman Data
(40768, 1)
---------------------------
Kinematics Data
(8192, 9)
2844
2804
2544
(410, 9) (5447, 9) (2335, 9)
---------------------------
Stock Data
(950, 10)
---------------------------
Computer Activity Data
(950, 22)
---------------------------
Ailerons Data
(7154, 41)
(6596, 41)
(358, 41) (6796, 41) (6596, 41)
---------------------------
Elevators Data
(8752, 19)
(7847, 19)
(438, 19) (8314, 19) (7847, 19)
---------------------------
Pole Telecomm Data
(5000, 49)
(10000, 49)
(438, 19) (8314, 19) (7847, 19)
---------------------------
(157, 7)
(119, 7)
(116, 7)


In [43]:
########################### TwoStagetrAdaBoostR2 #######################################
from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2

src_idx = np.arange(start=0, stop=(src_size_ailerons-1), step=1)
tgt_idx = np.arange(start=src_size_ailerons, stop=((src_size_ailerons + tgt_size_ailerons)-1), step=1)

#range(0, src_size_ailerons)
#range(src_size_ailerons, src_size_ailerons + tgt_size_ailerons)

def get_estimator(**kwargs):
    return DecisionTreeRegressor(max_depth=6)

model_TwoTrAda = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 100, fit_intercept=False)
model_TwoTrAda.fit(np_train_X, np_train_y_list, src_idx, tgt_idx)

y_pred_TwoTrAda = model_TwoTrAda.predict(AileronsData_df_test_X)
mse_TwoTrAda = sqrt(mean_squared_error(AileronsData_df_test_y, y_pred_TwoTrAda))
print("RMSE of Two-Stage TrAdaboost.R2:", mse_TwoTrAda)

# r2_score_twostageboost_values = pearsonr(AileronsData_df_test_y, y_pred_TwoTrAda)
# r2_score_twostageboost = (r2_score_twostageboost_values[0])**2
# print("R^2 of TrAdaboostR2:", r2_score_twostageboost)

RMSE of Two-Stage TrAdaboost.R2: 0.00020420204780042573


In [44]:
########################### AdaBoostR2 ###############################################
model_Ada = AdaBoostRegressor(DecisionTreeRegressor(max_depth=6), n_estimators = 100)
model_Ada.fit(np_train_X, np_train_y_list)

y_pred_ada = model_Ada.predict(AileronsData_df_test_X)
mse_ada = sqrt(mean_squared_error(AileronsData_df_test_y, y_pred_ada))
print("RMSE of Adaboost.R2:", mse_ada)

RMSE of Adaboost.R2: 0.0001772920248843519


In [ ]:
######################### Kernel Mean Matching #######################################
from adapt.instance_based import KMM

model_KMM = KMM(get_estimator = get_estimator)
model_KMM.fit(np_train_X, np_train_y_list, src_idx, tgt_idx)

print(model_KMM.weights_)

y_pred_TwoTrAda = model_TwoTrAda.predict(AileronsData_df_test_X)
mse_TwoTrAda = sqrt(mean_squared_error(AileronsData_df_test_y, y_pred_TwoTrAda))
print("RMSE of Two-Stage TrAdaboost.R2:", mse_TwoTrAda)

In [11]:
from sklearn.model_selection import GridSearchCV

model_TwoTrAda = TwoStageTrAdaBoostR2(get_estimator = get_estimator)
parameters_TwoTrAda = {'learning_rate': [0.01,0.02,0.03,0.04],
                    'n_estimators_fs'    : [10, 50, 100, 200],
                    'n_estimators' : [10,50,100, 200],
                    'cv'    : [4,6,8,10]
                     }

grid_TwoTrAda = GridSearchCV(estimator = model_TwoTrAda, param_grid = parameters_TwoTrAda, cv = 10, n_jobs=-1)
grid_TwoTrAda.fit(np_train_X, np_train_y_list, {src_idx, tgt_idx})

# Results from Grid Search
print(" Results from Grid Search " )
print()
print("\n The best estimator across ALL searched params:\n",
      grid_TwoTrAda.best_estimator_)
print("\n The best score across ALL searched params:\n",
      grid_TwoTrAda.best_score_)
print("\n The best parameters across ALL searched params:\n",
      grid_TwoTrAda.best_params_)


TypeError: unhashable type: 'numpy.ndarray'

In [6]:
#Using GridSearchCV
GBR = GradientBoostingRegressor()
parameters = {'learning_rate': [0.01,0.02,0.03,0.04],
              'subsample'    : [0.9, 0.5, 0.2, 0.1],
              'n_estimators' : [100,500,1000, 1500],
              'max_depth'    : [4,6,8,10]
             }
grid_GBR = GridSearchCV(estimator=GBR, param_grid = parameters, cv = 2, n_jobs=-1)
grid_GBR.fit(np_train_X, np_train_y_list)

# Results from Grid Search
print(" Results from Grid Search " )
print()
print("\n The best estimator across ALL searched params:\n",
      grid_GBR.best_estimator_)
print("\n The best score across ALL searched params:\n",
      grid_GBR.best_score_)
print("\n The best parameters across ALL searched params:\n",
      grid_GBR.best_params_)


 Results from Grid Search 


 The best estimator across ALL searched params:
 GradientBoostingRegressor(learning_rate=0.01, max_depth=4, n_estimators=1000,
                          subsample=0.5)

 The best score across ALL searched params:
 0.8474244872852686

 The best parameters across ALL searched params:
 {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 1000, 'subsample': 0.5}


In [7]:
ABR = AdaBoostRegressor()
parameters = {'learning_rate': [0.01,0.02,0.03,0.04],
              'subsample'    : [0.9, 0.5, 0.2, 0.1],
              'n_estimators' : [100,500,1000, 1500],
              'max_depth'    : [4,6,8,10]
             }
grid_ABR = GridSearchCV(estimator=GBR, param_grid = parameters, cv = 2, n_jobs=-1)
grid_ABR.fit(np_train_X, np_train_y_list)

# Results from Grid Search
print(" Results from Grid Search " )
print()
print("\n The best estimator across ALL searched params:\n",
      grid_ABR.best_estimator_)
print("\n The best score across ALL searched params:\n",
      grid_ABR.best_score_)
print("\n The best parameters across ALL searched params:\n",
      grid_ABR.best_params_)

 Results from Grid Search 


 The best estimator across ALL searched params:
 GradientBoostingRegressor(learning_rate=0.01, max_depth=8, n_estimators=500,
                          subsample=0.2)

 The best score across ALL searched params:
 0.8476612703679458

 The best parameters across ALL searched params:
 {'learning_rate': 0.01, 'max_depth': 8, 'n_estimators': 500, 'subsample': 0.2}


In [ ]:
print("hello")

In [105]:
from adapt.utils import toy_regression

X, y, src_index, tgt_index, tgt_index_labeled = toy_regression()